In [1]:
# NextPNR Parameter Sweep with MLIR Processing Pipeline
# Comprehensive tool for FPGA parameter optimization using MLIR-generated benchmarks

# Enable auto reload for imports
%load_ext autoreload
%autoreload 2

# Standard library imports
import subprocess
import os
import sys
import re
import time
import json
from pathlib import Path
from datetime import datetime
import itertools
import concurrent.futures

# Third-party imports
import pandas as pd
import numpy as np
from loguru import logger

# Configure loguru logger - output only to stdout (only if not already configured)
# logger.remove()  # Remove default handler
# logger.add(sys.stdout, format="{time} | {level} | {message}", level="INFO")

print("=== NextPNR Parameter Sweep with MLIR Processing Pipeline ===")
print("Comprehensive FPGA parameter optimization using MLIR-generated benchmarks")
print("Auto-reload enabled for module imports")

print("✓ Core libraries and logging configured")

=== NextPNR Parameter Sweep with MLIR Processing Pipeline ===
Comprehensive FPGA parameter optimization using MLIR-generated benchmarks
Auto-reload enabled for module imports
✓ Core libraries and logging configured


In [2]:
# Configuration and Parameter Setup
# Define parameter ranges, paths, and constants for the parameter sweep

import numpy as np
import os
from pathlib import Path

# Define parameter ranges for the sweep
CONNECTIVITY_FACTORS = np.arange(0.0, 2.0, 0.2)  # 0.0 to 2.0 with step 0.2
CONGESTION_FACTORS = np.arange(0.0, 2.0, 0.2)    # 0.0 to 2.0 with step 0.2

# Create parameter grid - use reduced set for testing or full grid for production
# grid = list(itertools.product(CONNECTIVITY_FACTORS, CONGESTION_FACTORS))  # Full grid
grid = [(0, 0), (0, 1), (1, 0), (1, 1)]  # Reduced grid for testing

# software dir configuration
CALYX_PATH: str = "/home/kelvin/calyx"
MLIR_OPT_PATH: str = "/home/kelvin/circt/build/vscode/bin/mlir-opt"
HLSTOOL_PATH: str = "/home/kelvin/circt/build/vscode/bin/hlstool"

# Project and directory configuration
MY_FAB_ROOT: Path = Path("/home/kelvin/FABulous_fork")
FAB_PROJ_DIR: Path = Path("/home/kelvin/FABulous_fork/myProject")

# Compilation results directory structure - Stage-based organization
COMPILATION_RESULT_DIR: Path = Path("/home/kelvin/FABulous_fork/myProject/PnR/compilation_result")

# Stage-specific directories for cleaner organization and simpler naming
MLIR_OPTIMIZED_DIR: Path = COMPILATION_RESULT_DIR / "01_optimized_mlir"
MLIR_EXTRACTED_DIR: Path = COMPILATION_RESULT_DIR / "02_extracted_mlir"
FUTIL_OUTPUT_DIR: Path = COMPILATION_RESULT_DIR / "03_futil"
VERILOG_OUTPUT_DIR: Path = COMPILATION_RESULT_DIR / "04_verilog"
OUTPUT_DIR: Path = Path("/home/kelvin/FABulous_fork/myProject/PnR/parameter_sweep_results")

# Benchmark directory configuration - MLIR files only
BENCHMARK_ROOT_DIR: Path = Path("/home/kelvin/FABulous_fork/myProject/PnR/mlir")

# Set up environment variables
os.environ["FAB_PROJ_DIR"] = str(FAB_PROJ_DIR)
os.environ["PATH"] = f"/home/kelvin/nextpnr/build/bba:{os.environ['PATH']}"
os.environ["PATH"] = f"/home/kelvin/nextpnr/build:{os.environ['PATH']}"
os.environ["PATH"] = f"/home/kelvin/yosys:{os.environ['PATH']}"
os.environ["PATH"] = f"/home/kelvin/circt/build/vscode/bin:{os.environ['PATH']}"
os.environ["CALYX_PRIMITIVES_DIR"] = str(Path(CALYX_PATH))

# Create necessary directories for all processing stages
COMPILATION_RESULT_DIR.mkdir(exist_ok=True)
MLIR_OPTIMIZED_DIR.mkdir(exist_ok=True)
MLIR_EXTRACTED_DIR.mkdir(exist_ok=True)
FUTIL_OUTPUT_DIR.mkdir(exist_ok=True)
VERILOG_OUTPUT_DIR.mkdir(exist_ok=True)
OUTPUT_DIR.mkdir(exist_ok=True)

# FABulous file paths
CHIPDB_PATH = FAB_PROJ_DIR / ".FABulous/hycube.bit"
JSON_INPUT = FAB_PROJ_DIR / "user_design/synth_test.json"
CONSTRAIN_PAIR = FAB_PROJ_DIR / ".FABulous/hycube_constrain_pair.inc"
FDC_PATH = FAB_PROJ_DIR / "user_design/test.fdc"

# Fallback source file if no generated Verilog files are available
FALLBACK_SOURCE_HDL = MY_FAB_ROOT / "benchmarks/userbench/loop_array_inner/loop_array_inner.sv"

# Fixed parameters for nextpnr runs
BETA_VALUE = 0.9
PLACE_TRIALS = 1
ROUTER_TIMEOUT = 20000

# Initialize available MLIR files and Verilog files structures
availableMlirFiles = []
availableVerilogFiles = []
benchmarkStructure = {}

print(f"Configuration setup complete:")
print(f"  Parameter combinations: {len(CONNECTIVITY_FACTORS)} × {len(CONGESTION_FACTORS)} = {len(CONNECTIVITY_FACTORS) * len(CONGESTION_FACTORS)} total")
print(f"  Test grid size: {len(grid)} combinations")
print(f"  Compilation results directory: {COMPILATION_RESULT_DIR}")
print(f"  Stage directories:")
print(f"    01_optimized_mlir/ - MLIR optimization results")
print(f"    02_extracted_mlir/ - Loop-to-function extraction results")
print(f"    03_intermediate_mlir/ - Individual function MLIR files")
print(f"    04_futil/ - Generated FUTIL files")
print(f"    05_verilog/ - Final Verilog output")
print(f"  Parameter sweep results directory: {OUTPUT_DIR}")
print(f"  Benchmark root directory: {BENCHMARK_ROOT_DIR}")
print(f"  Starting with unoptimized MLIR files from BENCHMARK folder")

print("✓ Configuration and paths set up successfully")

Configuration setup complete:
  Parameter combinations: 10 × 10 = 100 total
  Test grid size: 4 combinations
  Compilation results directory: /home/kelvin/FABulous_fork/myProject/PnR/compilation_result
  Stage directories:
    01_optimized_mlir/ - MLIR optimization results
    02_extracted_mlir/ - Loop-to-function extraction results
    03_intermediate_mlir/ - Individual function MLIR files
    04_futil/ - Generated FUTIL files
    05_verilog/ - Final Verilog output
  Parameter sweep results directory: /home/kelvin/FABulous_fork/myProject/PnR/parameter_sweep_results
  Benchmark root directory: /home/kelvin/FABulous_fork/myProject/PnR/mlir
  Starting with unoptimized MLIR files from BENCHMARK folder
✓ Configuration and paths set up successfully


In [3]:
# MLIR Processing Pipeline Configuration
# Set up the complete MLIR → FUTIL → Verilog processing pipeline starting with unoptimized MLIR files

import re
import os
from pathlib import Path
import subprocess


# MLIR Directory Configuration
MLIR_DIR = Path("mlir")

def findMlirBenchmarks(benchmarkDir: Path) -> list:
    """Find all .mlir files in the benchmark directory.

    Parameters
    ----------
    benchmarkDir : Path
        Path to the benchmark directory containing MLIR files.

    Returns
    -------
    list
        List of MLIR file paths found in the directory.
    """
    mlirFiles = []
    
    if benchmarkDir.exists():
        # Find all .mlir files directly in the benchmark directory
        mlirFiles = list(benchmarkDir.glob("*.mlir"))
        print(f"Found {len(mlirFiles)} MLIR files in {benchmarkDir}")
        for mlirFile in mlirFiles:
            print(f"  - {mlirFile.name}")
    else:
        logger.warning(f"Benchmark directory does not exist: {benchmarkDir}")
    
    return mlirFiles


print("MLIR processing configuration complete")
print("✓ Function extraction configured for unoptimized MLIR files")
print("Next: Define modular pipeline functions")

MLIR processing configuration complete
✓ Function extraction configured for unoptimized MLIR files
Next: Define modular pipeline functions


In [ ]:
# Direct pipeline functions that create result objects
import subprocess
import tempfile
import shutil

def run_verilog_pipeline(mlir_file: str) -> VerilogPipelineResult:
    """
    Run the complete MLIR to Verilog generation pipeline
    MLIR optimization -> Loop extraction -> FUTIL generation -> Verilog generation
    """
    start_time = time.time()
    
    result = VerilogPipelineResult(
        source_file=mlir_file,
        success=False,
        runtime=0.0,
        verilog_files=[]
    )
    
    try:
        # Stage 1: MLIR Optimization
        mlir_start = time.time()
        result.optimized_mlir_file, mlir_success, mlir_error = run_mlir_optimization(mlir_file)
        result.mlir_optimization_time = time.time() - mlir_start
        
        if not mlir_success:
            result.failure_stage = FailureType.MLIR_OPTIMIZATION
            result.error_message = mlir_error or "MLIR optimization failed"
            result.runtime = time.time() - start_time
            return result
        
        # Stage 2: Loop Extraction  
        loop_start = time.time()
        result.loops_found, loop_success, loop_error = extract_loops_from_mlir(result.optimized_mlir_file)
        result.loop_extraction_time = time.time() - loop_start
        
        if not loop_success:
            result.failure_stage = FailureType.LOOP_EXTRACTION
            result.error_message = loop_error or "Loop extraction failed"
            result.runtime = time.time() - start_time
            return result
        
        # Stage 3: FUTIL Generation
        futil_start = time.time()
        result.futil_file, futil_success, futil_error = generate_futil_from_mlir(result.optimized_mlir_file)
        result.futil_generation_time = time.time() - futil_start
        
        if not futil_success:
            result.failure_stage = FailureType.FUTIL_GENERATION
            result.error_message = futil_error or "FUTIL generation failed"
            result.runtime = time.time() - start_time
            return result
        
        # Stage 4: Verilog Generation
        result.verilog_files, verilog_success, verilog_error = generate_verilog_from_futil(result.futil_file)
        
        if not verilog_success:
            result.failure_stage = FailureType.VERILOG_GENERATION
            result.error_message = verilog_error or "Verilog generation failed"
            result.runtime = time.time() - start_time
            return result
            
        result.success = True
        
    except Exception as e:
        result.failure_stage = FailureType.VERILOG_GENERATION
        result.error_message = str(e)
    
    result.runtime = time.time() - start_time
    return result

def run_mlir_optimization(mlir_file: str) -> Tuple[Optional[str], bool, Optional[str]]:
    """Run MLIR optimization passes - returns (output_file, success, error_message)"""
    try:
        optimized_file = mlir_file.replace('.mlir', '_opt.mlir')
        
        cmd = [
            'mlir-opt',
            '--canonicalize',
            '--cse',
            '--loop-unroll',
            mlir_file,
            '-o', optimized_file
        ]
        
        result = subprocess.run(cmd, capture_output=True, text=True, timeout=60)
        
        if result.returncode == 0 and os.path.exists(optimized_file):
            return optimized_file, True, None
        else:
            error_msg = f"MLIR optimization failed: {result.stderr}"
            return None, False, error_msg
            
    except subprocess.TimeoutExpired:
        return None, False, "MLIR optimization timed out"
    except Exception as e:
        return None, False, f"MLIR optimization error: {str(e)}"

def extract_loops_from_mlir(mlir_file: str) -> Tuple[Optional[int], bool, Optional[str]]:
    """Extract loop information from MLIR file - returns (loop_count, success, error_message)"""
    try:
        with open(mlir_file, 'r') as f:
            content = f.read()
        
        # Simple loop detection - count scf.for and affine.for
        loop_count = content.count('scf.for') + content.count('affine.for')
        
        return loop_count, True, None
        
    except FileNotFoundError:
        return None, False, f"MLIR file not found: {mlir_file}"
    except Exception as e:
        return None, False, f"Loop extraction error: {str(e)}"

def generate_futil_from_mlir(mlir_file: str) -> Tuple[Optional[str], bool, Optional[str]]:
    """Generate FUTIL (.futil) file from optimized MLIR - returns (output_file, success, error_message)"""
    try:
        futil_file = mlir_file.replace('.mlir', '.futil').replace('_opt.futil', '.futil')
        
        # Using calyx frontend to convert MLIR to FUTIL
        cmd = [
            'calyx',
            '--frontend', 'mlir',
            '--backend', 'futil',
            mlir_file,
            '-o', futil_file
        ]
        
        result = subprocess.run(cmd, capture_output=True, text=True, timeout=120)
        
        if result.returncode == 0 and os.path.exists(futil_file):
            return futil_file, True, None
        else:
            error_msg = f"FUTIL generation failed: {result.stderr}"
            return None, False, error_msg
            
    except subprocess.TimeoutExpired:
        return None, False, "FUTIL generation timed out"
    except Exception as e:
        return None, False, f"FUTIL generation error: {str(e)}"

def generate_verilog_from_futil(futil_file: str) -> Tuple[List[str], bool, Optional[str]]:
    """Generate Verilog files from FUTIL - returns (verilog_files, success, error_message)"""
    try:
        output_dir = os.path.dirname(futil_file)
        base_name = os.path.splitext(os.path.basename(futil_file))[0]
        
        cmd = [
            'fud',
            'exec',
            '--from', 'futil',
            '--to', 'verilog',
            futil_file,
            '--output-dir', output_dir
        ]
        
        result = subprocess.run(cmd, capture_output=True, text=True, timeout=180)
        
        if result.returncode == 0:
            # Look for generated Verilog files
            verilog_files = []
            for file in os.listdir(output_dir):
                if file.startswith(base_name) and file.endswith('.v'):
                    verilog_files.append(os.path.join(output_dir, file))
            
            if verilog_files:
                return verilog_files, True, None
            else:
                return [], False, "No Verilog files generated"
        else:
            error_msg = f"Verilog generation failed: {result.stderr}"
            return [], False, error_msg
            
    except subprocess.TimeoutExpired:
        return [], False, "Verilog generation timed out"
    except Exception as e:
        return [], False, f"Verilog generation error: {str(e)}"

def run_synthesis(verilog_file: str) -> SynthesisResult:
    """Run FABulous synthesis preprocessing - returns SynthesisResult directly"""
    start_time = time.time()
    
    try:
        # Call the existing runParameterSweepForSource function for synthesis only
        synthesis_result, _ = runParameterSweepForSource(verilog_file)
        
        return SynthesisResult(
            success=synthesis_result.get('success', False),
            runtime=synthesis_result.get('runtime', time.time() - start_time),
            verilog_file=synthesis_result.get('verilog_file'),
            error_message=synthesis_result.get('error_message')
        )
        
    except Exception as e:
        return SynthesisResult(
            success=False,
            runtime=time.time() - start_time,
            error_message=f"Synthesis error: {str(e)}"
        )

def run_parameter_sweep(synthesis_result: SynthesisResult) -> List[ParameterResult]:
    """Run parameter sweep using synthesis result - returns list of ParameterResult directly"""
    if not synthesis_result.success or not synthesis_result.verilog_file:
        return []
    
    try:
        # Call the existing runParameterSweepForSource function for parameter sweep
        _, parameter_results = runParameterSweepForSource(synthesis_result.verilog_file)
        
        # Convert parameter dictionaries to ParameterResult objects
        converted_params = []
        for i, param_dict in enumerate(parameter_results):
            param_result = ParameterResult(
                config_id=i,
                success=param_dict.get('success', False),
                runtime=param_dict.get('runtime', 0.0),
                parameters=param_dict.get('parameters', {}),
                placement_info=param_dict.get('placement_info'),
                routing_info=param_dict.get('routing_info'),
                failure_type=FailureType(param_dict.get('failure_type', 'unknown')) if param_dict.get('failure_type') else None,
                error_message=param_dict.get('error_message')
            )
            converted_params.append(param_result)
        
        return converted_params
        
    except Exception as e:
        # Return a single failed parameter result
        return [ParameterResult(
            config_id=0,
            success=False,
            runtime=0.0,
            failure_type=FailureType.UNKNOWN,
            error_message=f"Parameter sweep error: {str(e)}"
        )]

def run_complete_pipeline(mlir_file: str) -> CompletePipelineResult:
    """
    Run the complete MLIR to hardware pipeline:
    MLIR -> Verilog -> Synthesis -> Parameter Sweep
    """
    start_time = time.time()
    
    # Stage 1: Verilog Generation Pipeline
    verilog_result = run_verilog_pipeline(mlir_file)
    
    # Stage 2: Synthesis (if Verilog generation succeeded)
    if verilog_result.success and verilog_result.verilog_files:
        primary_verilog = verilog_result.verilog_files[0]
        synthesis_result = run_synthesis(primary_verilog)
    else:
        synthesis_result = SynthesisResult(success=False, runtime=0.0)
    
    # Stage 3: Parameter Sweep (if synthesis succeeded)
    if synthesis_result.success:
        parameter_results = run_parameter_sweep(synthesis_result)
    else:
        parameter_results = []
    
    # Determine overall success and primary failure type
    overall_success = False
    primary_failure_type = FailureType.NONE
    
    if not verilog_result.success:
        primary_failure_type = verilog_result.failure_stage or FailureType.VERILOG_GENERATION
    elif not synthesis_result.success:
        primary_failure_type = FailureType.SYNTHESIS
    elif not parameter_results or not any(p.success for p in parameter_results):
        # Find most common parameter failure type
        if parameter_results:
            failure_types = [p.failure_type for p in parameter_results if p.failure_type]
            if failure_types:
                primary_failure_type = max(set(failure_types), key=failure_types.count)
            else:
                primary_failure_type = FailureType.UNKNOWN
        else:
            primary_failure_type = FailureType.UNKNOWN
    else:
        overall_success = True
        primary_failure_type = FailureType.NONE
    
    # Create and return complete result
    return CompletePipelineResult(
        source_file=mlir_file,
        overall_success=overall_success,
        total_runtime=time.time() - start_time,
        verilog_pipeline=verilog_result,
        synthesis_result=synthesis_result,
        parameter_results=parameter_results,
        primary_failure_type=primary_failure_type
    )

def process_mlir_benchmarks(mlir_files: List[str]) -> List[CompletePipelineResult]:
    """Process multiple MLIR files through the complete pipeline"""
    results = []
    
    print(f"Processing {len(mlir_files)} MLIR files through complete pipeline...")
    
    for i, mlir_file in enumerate(mlir_files):
        print(f"\n[{i+1}/{len(mlir_files)}] Processing: {os.path.basename(mlir_file)}")
        
        try:
            result = run_complete_pipeline(mlir_file)
            results.append(result)
            
            status = "✓ SUCCESS" if result.overall_success else f"✗ FAILED ({result.primary_failure_type.value})"
            print(f"  Result: {status} | Runtime: {result.total_runtime:.2f}s")
            
        except Exception as e:
            print(f"  ERROR: {e}")
            # Create failed result with proper structure
            failed_verilog_result = VerilogPipelineResult(
                source_file=mlir_file,
                success=False,
                runtime=0.0,
                failure_stage=FailureType.UNKNOWN,
                error_message=str(e)
            )
            
            failed_result = CompletePipelineResult(
                source_file=mlir_file,
                overall_success=False,
                total_runtime=0.0,
                verilog_pipeline=failed_verilog_result,
                synthesis_result=SynthesisResult(success=False, runtime=0.0),
                parameter_results=[],
                primary_failure_type=FailureType.UNKNOWN
            )
            results.append(failed_result)
    
    return results

In [ ]:
import inner_loop_to_func

def stepConvertInnerLoops(srcPath: Path, dstDir: Path) -> bool:
    """Step 2: Convert inner loops to functions with race condition protection.
    """

    print(f"Step 2: Converting inner loops to functions: {srcPath.name}")

    try:
        result = inner_loop_to_func.process_mlir_file(str(srcPath), str(dstDir))
    except Exception as e:
        print(f"Inner loop conversion failed for {srcPath.name}: {e}")
        return False
    
    return True


In [ ]:
def stepMlirToFutil(mlirPath: Path, futilPath: Path) -> list:
    """Step 3: Convert MLIR to Calyx FUTIL format using CIRCT with race condition protection.
    """
    print(f"Step 3: Converting MLIR to FUTIL: {mlirPath.name} -> multiple FUTIL files")
    
    generatedFutilFiles = []
    
    for funcPath in mlirPath.glob("*.mlir"):
        # Create clean FUTIL filename for each function 
        # For main functions: use baseName_funcName (e.g., nw_nw_nw_nw)
        # For inner loop functions: use funcName directly (e.g., nw_nw_inner_loop_1)
        
        funcFutilPath = futilPath.parent / f"{funcPath.stem}.futil"
        funcMlirPath = futilPath.parent / f"{funcPath.stem}.mlir"

        print(f"Processing function '{funcPath}' -> {funcFutilPath.name}")

        try:
            # Use retry mechanism for hlstool command
            hlstoolCmd = [
                "hlstool", 
                "--calyx-hw", 
                "--output-level=core", 
                "--ir", 
                "--allow-unregistered-dialects", 
                str(funcPath),
                "-o", str(funcMlirPath)
            ]

            result = subprocess.run(hlstoolCmd, capture_output=True, text=True, timeout=30)
            
            if result.returncode != 0:
                raise RuntimeError(f"hlstool failed: {result.stderr}")
                
            print(f"✓ hlstool completed for function '{funcPath}'")
            
            # Step 2: fud2 command to convert to FUTIL with retry
            translateCmd = [
                "circt-translate", 
                str(funcMlirPath), 
                "-o", str(funcFutilPath), 
                "--export-calyx"
            ]

            result = subprocess.run(translateCmd, capture_output=True, text=True, timeout=60)
            
            if result.returncode != 0:
                raise RuntimeError(f"circt-translate failed: {result.stderr}")
                
            print(f"circt-translate completed: {funcMlirPath.name} -> {funcFutilPath.name}")
            generatedFutilFiles.append(funcFutilPath)
            
            # Clean up intermediate MLIR file to save space
            funcMlirPath.unlink(missing_ok=True)
            
        except Exception as e:
            print(f"Error processing function '{funcPath}': {e}")
            # Clean up any partial files
            funcMlirPath.unlink(missing_ok=True)
            funcFutilPath.unlink(missing_ok=True)
            continue
    
    return generatedFutilFiles


In [ ]:

def stepFutilToVerilog(futilFiles: list, verilogBasePath: Path) -> list:
    """Step 4: Convert FUTIL files to Verilog using Calyx.
    
    This step processes multiple FUTIL files and generates corresponding Verilog files.

    Parameters
    ----------
    futilFiles : list
        List of FUTIL file paths to convert
    verilogBasePath : Path
        Base path for the output Verilog files (will be modified for each FUTIL file)

    Returns
    -------
    list
        List of successfully generated Verilog file paths, empty list if all failed
    """
    print(f"Step 4: Converting {len(futilFiles)} FUTIL files to Verilog")
    
    generatedVerilogFiles = []
    
    for futilPath in futilFiles:
        # Create simplified Verilog filename - use just the FUTIL file name
        verilogPath = verilogBasePath.parent / f"{futilPath.stem}.sv"
        
        # Calyx compilation command with proper library path
        calyxCmd = [
            "/home/kelvin/calyx/target/debug/calyx",
            "-l", "/home/kelvin/calyx",  # Add library path
            "-p", "fsm-opt",
            "-x", "simplify-with-control:without-register",
            "-x", "static-inline:offload-pause=false",
            "-p", "lower",
            "--nested",
            "-d", "papercut",
            "-d", "cell-share",
            "-o", str(verilogPath),
            "-b", "verilog",
            "--synthesis",
            str(futilPath)
        ]
        
        print(f"Running Calyx: {' '.join(calyxCmd)}")
        env = os.environ.copy()
        result = subprocess.run(calyxCmd, capture_output=True, text=True, timeout=300, cwd=CALYX_PATH, env=env)

        if result.returncode != 0:
            print(f"Calyx failed for {futilPath.name}: {result.stderr}")
            continue  # Skip this file, continue with others
        
        if not verilogPath.exists():
            print(f"Verilog file not created: {verilogPath}")
            continue  # Skip this file, continue with others
        
        generatedVerilogFiles.append(verilogPath)
        print(f"✓ Generated: {verilogPath.name}")
    
    if generatedVerilogFiles:
        print(f"✓ FUTIL to Verilog: {len(generatedVerilogFiles)}/{len(futilFiles)} files successful")
    else:
        print(f"✗ No Verilog files generated from {len(futilFiles)} FUTIL files")
    
    return generatedVerilogFiles


In [ ]:

def mlirPipeline(mlirSrc: Path, verilogDst: Path) -> list:
    """Master pipeline function that processes an unoptimized MLIR file through the complete pipeline.

    This function orchestrates the complete MLIR → FUTIL → Verilog pipeline:
    1. Apply MLIR optimizations to unoptimized MLIR file
    2. Convert inner loops to functions (optional)
    3. Convert MLIR to FUTIL using CIRCT (can generate multiple FUTIL files)
    4. Convert FUTIL to Verilog using Calyx (generates multiple Verilog files)

    Parameters
    ----------
    mlirSrc : Path
        Path to the unoptimized MLIR file
    verilogDst : Path
        Base path for the output Verilog files (will be modified for each function)

    Returns
    -------
    list
        List of successfully generated Verilog file paths, empty list if pipeline failed
    """
    print(f"=== Starting MLIR Pipeline: {mlirSrc.name} ===")
    
    # Validate input
    if not mlirSrc.exists():
        print(f"MLIR file does not exist: {mlirSrc}")
        return []
    
    if not mlirSrc.suffix == ".mlir":
        print(f"Source file must be a .mlir file: {mlirSrc}")
        return []
    
    # Create intermediate file paths using stage-specific directories for cleaner naming
    baseName = mlirSrc.stem
    
    # Stage 1: Optimized MLIR files
    optimizedMlirPath = MLIR_OPTIMIZED_DIR / f"{baseName}.mlir"
    
    # Stage 2: Extracted MLIR files (with loop-to-function conversion)
    extractedMlirDir = MLIR_EXTRACTED_DIR
    
    # Stage 3: Base path for FUTIL files (actual files will have function names)
    futilBasePath = FUTIL_OUTPUT_DIR / f"{baseName}.futil"
    
    # Ensure output directories exist
    verilogDst.parent.mkdir(parents=True, exist_ok=True)
    MLIR_OPTIMIZED_DIR.mkdir(parents=True, exist_ok=True)
    MLIR_EXTRACTED_DIR.mkdir(parents=True, exist_ok=True)
    FUTIL_OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
    
    try:
        # Step 1: Optimize MLIR
        if not stepOptimizeMlir(mlirSrc, optimizedMlirPath):
            print(f"Pipeline failed at step 1 (optimize MLIR) for {mlirSrc.name}")
            return []
        
        # Step 2: Convert inner loops (optional - continue if this fails)
        if not stepConvertInnerLoops(optimizedMlirPath, extractedMlirDir):
            print(f"Pipeline failed at step 2 (convert inner loops) for {mlirSrc.name}")
            return []

        # Step 3: MLIR to FUTIL (can generate multiple FUTIL files)
        futilFiles = stepMlirToFutil(extractedMlirDir, futilBasePath)
        if not futilFiles:
            print(f"Pipeline failed at step 3 (MLIR to FUTIL) for {mlirSrc.name}")
            return []
        
        # Step 4: FUTIL to Verilog (processes multiple FUTIL files) - output goes to Verilog directory
        verilogFiles = stepFutilToVerilog(futilFiles, verilogDst)
        if not verilogFiles:
            print(f"Pipeline failed at step 4 (FUTIL to Verilog) for {mlirSrc.name}")
            return []
        
        
        print(f"✅ Pipeline completed: {mlirSrc.name} → {len(verilogFiles)} Verilog files")
        return verilogFiles
        
    except Exception as e:
        print(f"Pipeline failed with exception for {mlirSrc.name}: {e}")
        return []


In [9]:

def processBenchmarkMlir(mlirFile: Path, verilogOutputDir: Path) -> list:
    """Process a single MLIR benchmark file using the pipeline.

    Parameters
    ----------
    mlirFile : Path
        Path to the MLIR benchmark file
    verilogOutputDir : Path
        Directory to store the generated Verilog files

    Returns
    -------
    list
        List of successfully generated Verilog file paths
    """
    benchmarkName = mlirFile.stem
    
    # Generate base output Verilog filename in the Verilog output directory
    verilogBaseFileName = f"{benchmarkName}.sv"
    verilogOutputBasePath = verilogOutputDir / verilogBaseFileName
    
    # Run the complete pipeline (returns list of generated Verilog files)
    pipelineResults = mlirPipeline(mlirFile, verilogOutputBasePath)
    return pipelineResults

def processAllMlirBenchmarks() -> list:
    """Process all MLIR benchmarks using the pipeline.

    Returns
    -------
    list
        List of all successfully generated Verilog file paths
    """
    print("Starting MLIR pipeline processing for all benchmarks...")
    
    # Find all MLIR files in the benchmark directory
    mlirFiles = findMlirBenchmarks(BENCHMARK_ROOT_DIR)
    if not mlirFiles:
        logger.warning(f"No MLIR files found in {BENCHMARK_ROOT_DIR}")
        return []
    
    # Create output directories for all processing stages
    VERILOG_OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
    MLIR_OPTIMIZED_DIR.mkdir(parents=True, exist_ok=True)
    MLIR_EXTRACTED_DIR.mkdir(parents=True, exist_ok=True)
    FUTIL_OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
    
    print(f"Found {len(mlirFiles)} MLIR files to process")
    print(f"Output structure:")
    print(f"  Verilog files: {VERILOG_OUTPUT_DIR}")
    print(f"  Stage-based intermediate files in compilation_result/01-05 directories")
    
    allGeneratedVerilog = []
    for mlirFile in mlirFiles:
        generatedFiles = processBenchmarkMlir(mlirFile, verilogOutputDir)
        allGeneratedVerilog.extend(generatedFiles)
    
    print(f"MLIR pipeline completed: {len(allGeneratedVerilog)} Verilog files generated from {len(mlirFiles)} MLIR files")

    return allGeneratedVerilog

print("MLIR pipeline functions defined")
print("✓ Master function: mlirPipeline(mlirSrc: Path, verilogDst: Path) -> list")
print("✓ Individual steps: stepOptimizeMlir, stepConvertInnerLoops, stepMlirToFutil, stepFutilToVerilog")
print("✓ Batch processing: processBenchmarkMlir, processAllMlirBenchmarks")
print(f"✓ Starting with unoptimized MLIR files from: {BENCHMARK_ROOT_DIR}")
print(f"✓ Output structure: compilation_result/01-05 stage directories")

MLIR pipeline functions defined
✓ Master function: mlirPipeline(mlirSrc: Path, verilogDst: Path) -> list
✓ Individual steps: stepOptimizeMlir, stepConvertInnerLoops, stepMlirToFutil, stepFutilToVerilog
✓ Batch processing: processBenchmarkMlir, processAllMlirBenchmarks
✓ Starting with unoptimized MLIR files from: /home/kelvin/FABulous_fork/myProject/PnR/mlir
✓ Output structure: compilation_result/01-05 stage directories


In [ ]:
# Improved Parallel Processing with Race Condition Prevention
# Enhanced parallel MLIR processing with better error handling and synchronization

import concurrent.futures
import threading
from typing import Callable, List, Tuple, Any

def processWithSafeParallel(
    items: List[Path], 
    processFunc: Callable[[Path], Any], 
    maxWorkers: int = 4,
    description: str = "Processing"
) -> List[Tuple[Path, Any, bool]]:
    """Process a list of items in parallel with race condition protection.
    
    Parameters
    ----------
    items : List[Path]
        List of items to process
    processFunc : Callable
        Function to apply to each item
    maxWorkers : int
        Maximum number of parallel workers
    description : str
        Description for logging
    
    Returns
    -------
    List[Tuple[Path, any, bool]]
        List of (item, result, success) tuples
    """
    results = []
    completed = 0
    
    print(f"Starting {description} for {len(items)} items with {maxWorkers} workers...")
    
    # Use a lock to coordinate file system access
    processLock = threading.Lock()
    
    def safeProcessItem(item: Path) -> Tuple[Path, any, bool]:
        """Safely process a single item with error handling."""
        nonlocal completed
        
        try:
            # Add small random delay to reduce simultaneous access
            import random
            time.sleep(random.uniform(0.1, 0.3))
            
            # Process the item
            result = processFunc(item)
            
            # Thread-safe logging
            with processLock:
                completed += 1
                print(f"[{completed}/{len(items)}] ✅ {item.name} completed")
                
            return (item, result, True)
            
        except Exception as e:
            with processLock:
                completed += 1
                print(f"[{completed}/{len(items)}] ❌ {item.name} failed: {e}")
                
            return (item, None, False)
    
    # Process in parallel with controlled concurrency
    with concurrent.futures.ThreadPoolExecutor(max_workers=maxWorkers) as executor:
        # Submit all tasks
        futures = {executor.submit(safeProcessItem, item): item for item in items}
        
        # Collect results as they complete
        for future in concurrent.futures.as_completed(futures):
            result = future.result()
            results.append(result)
    
    successCount = sum(1 for _, _, success in results if success)
    print(f"✅ {description} completed: {successCount}/{len(items)} successful")
    
    return results

def processAllMlirBenchmarksParallel(maxWorkers: int = 2) -> list:
    """Process all MLIR benchmarks using parallel processing with race condition prevention.
    
    Parameters
    ---------- 
    maxWorkers : int
        Maximum number of parallel workers (keep low to avoid resource conflicts)

    Returns
    -------
    list
        List of all successfully generated Verilog file paths
    """
    print(f"Starting parallel MLIR pipeline processing with {maxWorkers} workers...")
    
    # Find all MLIR files in the benchmark directory
    mlirFiles = findMlirBenchmarks(BENCHMARK_ROOT_DIR)
    if not mlirFiles:
        logger.warning(f"No MLIR files found in {BENCHMARK_ROOT_DIR}")
        return []
    
    # Create output directories for all processing stages
    VERILOG_OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
    MLIR_OPTIMIZED_DIR.mkdir(parents=True, exist_ok=True)
    MLIR_EXTRACTED_DIR.mkdir(parents=True, exist_ok=True)
    FUTIL_OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
    
    def processOneMlirFile(mlirFile: Path) -> list:
        """Process a single MLIR file through the complete pipeline."""
        try:
            verilogOutputBasePath = VERILOG_OUTPUT_DIR / f"{mlirFile.stem}.sv"
            generatedFiles = mlirPipeline(mlirFile, verilogOutputBasePath)
            return generatedFiles if generatedFiles else []
        except Exception as e:
            print(f"Pipeline failed for {mlirFile.name}: {e}")
            return []
    
    # Process all MLIR files in parallel
    results = processWithSafeParallel(
        mlirFiles,
        processOneMlirFile,
        maxWorkers=maxWorkers,
        description="MLIR Pipeline Processing"
    )
    
    # Collect all generated Verilog files
    allGeneratedVerilog = []
    for mlirFile, verilogFiles, success in results:
        if success and verilogFiles:
            allGeneratedVerilog.extend(verilogFiles)
    
    print(f"\n📊 Parallel MLIR Pipeline Summary:")
    print(f"   📁 Total MLIR files processed: {len(mlirFiles)}")
    print(f"   ✅ Successful conversions: {sum(1 for _, _, success in results if success)}")
    print(f"   ❌ Failed conversions: {sum(1 for _, _, success in results if not success)}")
    print(f"   📝 Total Verilog files generated: {len(allGeneratedVerilog)}")
    

    return allGeneratedVerilog

print("✅ Improved parallel processing with race condition prevention ready")
print("✅ Safe concurrent MLIR pipeline processing available")

✅ Improved parallel processing with race condition prevention ready
✅ Safe concurrent MLIR pipeline processing available


In [ ]:
# Execute MLIR Processing Pipeline and Organize Results - Sequential Processing

print("Starting MLIR to Verilog conversion process from unoptimized MLIR files...")
print("Processing will happen sequentially, file by file, for better control and debugging")

# Process MLIR benchmarks from the BENCHMARK folder
print(f"Processing unoptimized MLIR files from: {BENCHMARK_ROOT_DIR}")

# Find available MLIR files
availableMlirFiles = findMlirBenchmarks(BENCHMARK_ROOT_DIR)

if availableMlirFiles:
    print(f"Found {len(availableMlirFiles)} MLIR benchmark files:")
    for mlirFile in availableMlirFiles:
        print(f"  - {mlirFile.name}")
    
    # Process MLIR files sequentially with detailed progress tracking
    print(f"\n🔄 Starting sequential processing of {len(availableMlirFiles)} MLIR files...")
    
    availableVerilogFiles = []
    successfulFiles = 0
    failedFiles = 0
    
    # Create output directories
    VERILOG_OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
    MLIR_OPTIMIZED_DIR.mkdir(parents=True, exist_ok=True)
    MLIR_EXTRACTED_DIR.mkdir(parents=True, exist_ok=True)
    FUTIL_OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
    
    for fileIndex, mlirFile in enumerate(availableMlirFiles, 1):
        print(f"\n📁 [{fileIndex}/{len(availableMlirFiles)}] Processing: {mlirFile.name} ({fileIndex/len(availableMlirFiles)*100:.1f}%)")
        
        try:
            # Process this single MLIR file
            generatedFiles = processBenchmarkMlir(mlirFile, VERILOG_OUTPUT_DIR)
            
            if generatedFiles:
                availableVerilogFiles.extend(generatedFiles)
                successfulFiles += 1
                print(f"   ✅ Success: Generated {len(generatedFiles)} Verilog files")
                # Only log individual files if there are multiple functions per MLIR file
                if len(generatedFiles) > 1:
                    for vFile in generatedFiles:
                        print(f"      📝 {vFile.name}")
                else:
                    print(f"      📝 {generatedFiles[0].name}")
            else:
                failedFiles += 1
                print(f"   ❌ Failed: No Verilog files generated")
                
        except Exception as e:
            failedFiles += 1
            print(f"   💥 Exception: {e}")
            import traceback
            print(f"   📋 Traceback: {traceback.format_exc()}")
        
        # Brief pause between files for system stability
        if fileIndex < len(availableMlirFiles):
            import time
            time.sleep(0.5)  # Small delay between files
    
    # Summary of sequential processing
    print(f"\n📊 Sequential MLIR Processing Summary:")
    print(f"   📁 Total MLIR files processed: {len(availableMlirFiles)}")
    print(f"   ✅ Successful conversions: {successfulFiles}")
    print(f"   ❌ Failed conversions: {failedFiles}")
    print(f"   📝 Total Verilog files generated: {len(availableVerilogFiles)}")
    print(f"   📈 Success rate: {successfulFiles/len(availableMlirFiles)*100:.1f}%")
    
else:
    logger.warning(f"No MLIR files found in {BENCHMARK_ROOT_DIR}")
    # Check if we already have generated Verilog files
    if VERILOG_OUTPUT_DIR.exists():
        availableVerilogFiles = list(VERILOG_OUTPUT_DIR.glob("*.sv"))
        print(f"Found {len(availableVerilogFiles)} existing Verilog files")
    else:
        availableVerilogFiles = []
        logger.warning("No Verilog files found")

# Find Verilog Files Directly from VERILOG_OUTPUT_DIR

print("Finding .sv files directly from VERILOG_OUTPUT_DIR...")
print(f"Searching directory: {VERILOG_OUTPUT_DIR}")

# Initialize variables
availableVerilogFiles = []
benchmarkStructure = {}

# Check if VERILOG_OUTPUT_DIR exists and find all .sv files
if VERILOG_OUTPUT_DIR.exists():
    # Find all .sv files directly in the VERILOG_OUTPUT_DIR
    availableVerilogFiles = list(VERILOG_OUTPUT_DIR.glob("*.sv"))
    print(f"Found {len(availableVerilogFiles)} Verilog files:")
    
    for vFile in availableVerilogFiles:
        print(f"  📝 {vFile.name}")
    
    # Organize files for processing - group by benchmark name
    print(f"\n🗂️  Organizing Verilog files by benchmark source...")
    
    if availableVerilogFiles:
        # Group files by benchmark name for organization
        for vFile in availableVerilogFiles:
            # Extract the base benchmark name (before any function suffix)
            # Expected format: benchmarkName_functionName.sv
            baseName = vFile.stem
            
            # If there are function suffixes (like _funcName), extract the benchmark name
            if "_" in baseName:
                # Split by underscore and try to identify the benchmark part
                parts = baseName.split("_")
                # Use the first part as the benchmark name, or find a reasonable split
                benchmarkBaseName = parts[0]
                # If we have a pattern like "benchmark_func_name", use first two parts
                if len(parts) > 2 and parts[-1].isdigit():
                    benchmarkBaseName = "_".join(parts[:-1])
                elif len(parts) > 1:
                    benchmarkBaseName = parts[0]
            else:
                benchmarkBaseName = baseName
            
            if benchmarkBaseName not in benchmarkStructure:
                benchmarkStructure[benchmarkBaseName] = []
            benchmarkStructure[benchmarkBaseName].append(vFile)
        
        # Sort for consistent ordering
        for benchmarkName in benchmarkStructure:
            benchmarkStructure[benchmarkName].sort()

        print(f"Generated Verilog files organized into {len(benchmarkStructure)} benchmark groups:")
        for groupName, files in benchmarkStructure.items():
            print(f"  📊 {groupName}: {len(files)} files")
            # Only show individual files if there are multiple files per group
            if len(files) <= 3:  # Show individual files only for small groups
                for vFile in files:
                    print(f"    📝 {vFile.name}")
            else:
                print(f"    📝 {files[0].name} ... and {len(files)-1} more")

        print(f"\n📈 Final Results:")
        print(f"   📁 Total available Verilog files: {len(availableVerilogFiles)}")
        print(f"   🗂️  Organized into {len(benchmarkStructure)} benchmark groups")
        print("✅ Ready to proceed with parameter sweep using existing Verilog files")

    else:
        logger.warning("❌ No .sv files found in VERILOG_OUTPUT_DIR")
        print("🔍 Contents of VERILOG_OUTPUT_DIR:")
        if VERILOG_OUTPUT_DIR.exists():
            for item in VERILOG_OUTPUT_DIR.iterdir():
                print(f"  - {item.name}")
else:
    print(f"❌ VERILOG_OUTPUT_DIR does not exist: {VERILOG_OUTPUT_DIR}")
    availableVerilogFiles = []
    benchmarkStructure = {}
    
if not availableVerilogFiles:
    logger.warning("❌ No Verilog files found! Check VERILOG_OUTPUT_DIR and ensure it contains .sv files.")
    print("🔍 Diagnostic information:")
    print(f"   📂 VERILOG_OUTPUT_DIR: {VERILOG_OUTPUT_DIR}")
    print(f"   📂 VERILOG_OUTPUT_DIR exists: {VERILOG_OUTPUT_DIR.exists()}")
    # Fallback to original source if no generated files exist
    print(f"   🔄 Will use fallback source: {FALLBACK_SOURCE_HDL}")
else:
    print("✅ Verilog file detection completed successfully")
    print("🚀 Ready to proceed with parameter sweep using found Verilog files")

Starting MLIR to Verilog conversion process from unoptimized MLIR files...
Processing will happen sequentially, file by file, for better control and debugging
Processing unoptimized MLIR files from: /home/kelvin/FABulous_fork/myProject/PnR/mlir
Found 13 MLIR files in /home/kelvin/FABulous_fork/myProject/PnR/mlir
  - nw_nw.mlir
  - gemm_ncubed.mlir
  - sort_radix.mlir
  - fft_transpose.mlir
  - kmp_kmp.mlir
  - spmv_crs.mlir
  - gemm_blocked.mlir
  - fft_strided.mlir
  - md_knn.mlir
  - spmv_ellpack.mlir
  - stencil_stencil3d.mlir
  - bfs_queue.mlir
  - stencil_stencil2d.mlir
Found 13 MLIR benchmark files:
  - nw_nw.mlir
  - gemm_ncubed.mlir
  - sort_radix.mlir
  - fft_transpose.mlir
  - kmp_kmp.mlir
  - spmv_crs.mlir
  - gemm_blocked.mlir
  - fft_strided.mlir
  - md_knn.mlir
  - spmv_ellpack.mlir
  - stencil_stencil3d.mlir
  - bfs_queue.mlir
  - stencil_stencil2d.mlir

🔄 Starting sequential processing of 13 MLIR files...

📁 [1/13] Processing: nw_nw.mlir (7.7%)
=== Starting MLIR Pipelin

2025-06-27 21:13:39.467 | ERROR    | __main__:stepMlirToFutil:57 - Error processing function '/home/kelvin/FABulous_fork/myProject/PnR/compilation_result/02_extracted_mlir/nw_nw_inner_loop_2.mlir': hlstool failed: /home/kelvin/FABulous_fork/myProject/PnR/compilation_result/02_extracted_mlir/nw_nw_inner_loop_2.mlir:22:11: error: 'calyx.comb_group' op with port: %arg_mem_0.read_data. This operation is not combinational.
    %13 = arith.cmpi eq, %12, %8 : i32
          ^
/home/kelvin/FABulous_fork/myProject/PnR/compilation_result/02_extracted_mlir/nw_nw_inner_loop_2.mlir:22:11: note: see current operation: 
"calyx.comb_group"() <{sym_name = "bb0_20"}> ({
  "calyx.assign"(%19#0, %20#3) : (i32, i32) -> ()
  "calyx.assign"(%19#1, %24#2) : (i32, i32) -> ()
  "calyx.assign"(%20#0, %21#2) : (i1, i1) -> ()
  "calyx.assign"(%21#0, %30#2) : (i32, i32) -> ()
  "calyx.assign"(%30#0, %31#4) : (i32, i32) -> ()
  "calyx.assign"(%30#1, %36#3) : (i32, i32) -> ()
  "calyx.assign"(%36#0, %37#2) : (i1, i1) 

✓ hlstool completed for function '/home/kelvin/FABulous_fork/myProject/PnR/compilation_result/02_extracted_mlir/nw_nw_inner_loop_4.mlir'
circt-translate completed: nw_nw_inner_loop_4.mlir -> nw_nw_inner_loop_4.futil
Processing function '/home/kelvin/FABulous_fork/myProject/PnR/compilation_result/02_extracted_mlir/nw_nw_inner_loop_1.mlir' -> nw_nw_inner_loop_1.futil
✓ hlstool completed for function '/home/kelvin/FABulous_fork/myProject/PnR/compilation_result/02_extracted_mlir/nw_nw_inner_loop_1.mlir'
circt-translate completed: nw_nw_inner_loop_1.mlir -> nw_nw_inner_loop_1.futil
Processing function '/home/kelvin/FABulous_fork/myProject/PnR/compilation_result/02_extracted_mlir/nw_nw_inner_loop_2.mlir' -> nw_nw_inner_loop_2.futil
Step 4: Converting 5 FUTIL files to Verilog
Running Calyx: /home/kelvin/calyx/target/debug/calyx -l /home/kelvin/calyx -p fsm-opt -x simplify-with-control:without-register -x static-inline:offload-pause=false -p lower --nested -d papercut -d cell-share -o /home/ke

2025-06-27 21:13:40.623 | ERROR    | __main__:stepMlirToFutil:57 - Error processing function '/home/kelvin/FABulous_fork/myProject/PnR/compilation_result/02_extracted_mlir/nw_nw_inner_loop_2.mlir': hlstool failed: /home/kelvin/FABulous_fork/myProject/PnR/compilation_result/02_extracted_mlir/nw_nw_inner_loop_2.mlir:22:11: error: 'calyx.comb_group' op with port: %arg_mem_0.read_data. This operation is not combinational.
    %13 = arith.cmpi eq, %12, %8 : i32
          ^
/home/kelvin/FABulous_fork/myProject/PnR/compilation_result/02_extracted_mlir/nw_nw_inner_loop_2.mlir:22:11: note: see current operation: 
"calyx.comb_group"() <{sym_name = "bb0_20"}> ({
  "calyx.assign"(%19#0, %20#3) : (i32, i32) -> ()
  "calyx.assign"(%19#1, %24#2) : (i32, i32) -> ()
  "calyx.assign"(%20#0, %21#2) : (i1, i1) -> ()
  "calyx.assign"(%21#0, %30#2) : (i32, i32) -> ()
  "calyx.assign"(%30#0, %31#4) : (i32, i32) -> ()
  "calyx.assign"(%30#1, %36#3) : (i32, i32) -> ()
  "calyx.assign"(%36#0, %37#2) : (i1, i1) 

circt-translate completed: gemm_ncubed_inner_loop_0.mlir -> gemm_ncubed_inner_loop_0.futil
Processing function '/home/kelvin/FABulous_fork/myProject/PnR/compilation_result/02_extracted_mlir/nw_nw_inner_loop_1.mlir' -> nw_nw_inner_loop_1.futil
✓ hlstool completed for function '/home/kelvin/FABulous_fork/myProject/PnR/compilation_result/02_extracted_mlir/nw_nw_inner_loop_1.mlir'
circt-translate completed: nw_nw_inner_loop_1.mlir -> nw_nw_inner_loop_1.futil
Processing function '/home/kelvin/FABulous_fork/myProject/PnR/compilation_result/02_extracted_mlir/nw_nw_inner_loop_2.mlir' -> nw_nw_inner_loop_2.futil
Step 4: Converting 6 FUTIL files to Verilog
Running Calyx: /home/kelvin/calyx/target/debug/calyx -l /home/kelvin/calyx -p fsm-opt -x simplify-with-control:without-register -x static-inline:offload-pause=false -p lower --nested -d papercut -d cell-share -o /home/kelvin/FABulous_fork/myProject/PnR/compilation_result/04_verilog/nw_nw_inner_loop_5.sv -b verilog --synthesis /home/kelvin/FABu

...

Processing function '/home/kelvin/FABulous_fork/myProject/PnR/compilation_result/02_extracted_mlir/fft_transpose_inner_loop_5.mlir' -> fft_transpose_inner_loop_5.futil
✓ hlstool completed for function '/home/kelvin/FABulous_fork/myProject/PnR/compilation_result/02_extracted_mlir/fft_transpose_inner_loop_5.mlir'
circt-translate completed: fft_transpose_inner_loop_5.mlir -> fft_transpose_inner_loop_5.futil
Processing function '/home/kelvin/FABulous_fork/myProject/PnR/compilation_result/02_extracted_mlir/stencil_stencil3d_inner_loop_3.mlir' -> stencil_stencil3d_inner_loop_3.futil
✓ hlstool completed for function '/home/kelvin/FABulous_fork/myProject/PnR/compilation_result/02_extracted_mlir/stencil_stencil3d_inner_loop_3.mlir'
circt-translate completed: stencil_stencil3d_inner_loop_3.mlir -> stencil_stencil3d_inner_loop_3.futil
Processing function '/home/kelvin/FABulous_fork/myProject/PnR/compilation_result/02_extracted_mlir/gemm_blocked_inner_loop_0.mlir' -> gemm_blocked_inner_loop_0.futi

2025-06-27 21:16:44.638 | ERROR    | __main__:stepMlirToFutil:57 - Error processing function '/home/kelvin/FABulous_fork/myProject/PnR/compilation_result/02_extracted_mlir/fft_transpose_inner_loop_4.mlir': hlstool failed: /home/kelvin/FABulous_fork/myProject/PnR/compilation_result/02_extracted_mlir/fft_transpose_inner_loop_4.mlir:10:10: error: Dialect `math' not found for custom op 'math.cos' 
    %5 = math.cos %4 : f64
         ^
/home/kelvin/FABulous_fork/myProject/PnR/compilation_result/02_extracted_mlir/fft_transpose_inner_loop_4.mlir:10:10: note: Registered dialects: affine, arith, builtin, calyx, cf, comb, esi, func, handshake, hw, memref, scf, seq, sv ; for more info on dialect registration see https://mlir.llvm.org/getting_started/Faq/#registered-loaded-dependent-whats-up-with-dialects-management



✓ hlstool completed for function '/home/kelvin/FABulous_fork/myProject/PnR/compilation_result/02_extracted_mlir/fft_transpose_inner_loop_7.mlir'
circt-translate completed: fft_transpose_inner_loop_7.mlir -> fft_transpose_inner_loop_7.futil
Processing function '/home/kelvin/FABulous_fork/myProject/PnR/compilation_result/02_extracted_mlir/fft_transpose_inner_loop_4.mlir' -> fft_transpose_inner_loop_4.futil
Processing function '/home/kelvin/FABulous_fork/myProject/PnR/compilation_result/02_extracted_mlir/spmv_ellpack_inner_loop_0.mlir' -> spmv_ellpack_inner_loop_0.futil
✓ hlstool completed for function '/home/kelvin/FABulous_fork/myProject/PnR/compilation_result/02_extracted_mlir/spmv_ellpack_inner_loop_0.mlir'
circt-translate completed: spmv_ellpack_inner_loop_0.mlir -> spmv_ellpack_inner_loop_0.futil
Step 4: Converting 39 FUTIL files to Verilog
Running Calyx: /home/kelvin/calyx/target/debug/calyx -l /home/kelvin/calyx -p fsm-opt -x simplify-with-control:without-register -x static-inline

2025-06-27 21:16:44.791 | ERROR    | __main__:stepFutilToVerilog:48 - Calyx failed for fft_transpose_inner_loop_8.futil: Error: /home/kelvin/FABulous_fork/myProject/PnR/compilation_result/03_futil/fft_transpose_inner_loop_8.futil
30 |    cst_2 = std_float_const(0, 64, -1.000000);
   |                                   ^ Parse error
expected bitwidth or float

2025-06-27 21:16:44.875 | ERROR    | __main__:stepFutilToVerilog:48 - Calyx failed for nw_nw_inner_loop_3.futil: Error: /home/kelvin/FABulous_fork/myProject/PnR/compilation_result/03_futil/nw_nw_inner_loop_3.futil
85 |      std_signext_1.in = load_3_reg.out;
   |      ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^ Malformed Structure: Obviously conflicting assignments found
...
83 |      std_signext_1.in = load_3_reg.out;
   |      ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^ `std_signext_1.in` is also written to here
...
257 |            else_br_0;
    |            ^^^^^^^^^^ Assignments activated by group enable, causing the conflict



✓ Generated: stencil_stencil3d_inner_loop_1.sv
Running Calyx: /home/kelvin/calyx/target/debug/calyx -l /home/kelvin/calyx -p fsm-opt -x simplify-with-control:without-register -x static-inline:offload-pause=false -p lower --nested -d papercut -d cell-share -o /home/kelvin/FABulous_fork/myProject/PnR/compilation_result/04_verilog/fft_strided_inner_loop_0.sv -b verilog --synthesis /home/kelvin/FABulous_fork/myProject/PnR/compilation_result/03_futil/fft_strided_inner_loop_0.futil
✓ Generated: fft_strided_inner_loop_0.sv
Running Calyx: /home/kelvin/calyx/target/debug/calyx -l /home/kelvin/calyx -p fsm-opt -x simplify-with-control:without-register -x static-inline:offload-pause=false -p lower --nested -d papercut -d cell-share -o /home/kelvin/FABulous_fork/myProject/PnR/compilation_result/04_verilog/fft_transpose_inner_loop_8.sv -b verilog --synthesis /home/kelvin/FABulous_fork/myProject/PnR/compilation_result/03_futil/fft_transpose_inner_loop_8.futil
Running Calyx: /home/kelvin/calyx/target

## 3. NextPNR Execution and Analysis Functions

Define functions to run FABulous preprocessing and nextpnr-himbaechel with comprehensive error analysis.

In [ ]:
# Enhanced data structures for complete pipeline
from dataclasses import dataclass
from typing import List, Optional, Dict, Any, Tuple
from enum import Enum
import time
import os

class FailureType(Enum):
    NONE = "none"
    MLIR_OPTIMIZATION = "mlir_optimization"
    LOOP_EXTRACTION = "loop_extraction" 
    FUTIL_GENERATION = "futil_generation"
    VERILOG_GENERATION = "verilog_generation"
    SYNTHESIS = "synthesis"
    PLACEMENT = "placement"
    ROUTING = "routing"
    UNKNOWN = "unknown"

@dataclass
class VerilogPipelineResult:
    """Unified MLIR to Verilog generation pipeline result"""
    source_file: str
    success: bool
    runtime: float
    
    # Output files
    optimized_mlir_file: Optional[str] = None
    futil_file: Optional[str] = None
    verilog_files: List[str] = None
    
    # Stage metrics
    mlir_optimization_time: float = 0.0
    loop_extraction_time: float = 0.0
    futil_generation_time: float = 0.0
    
    # Extracted information
    loops_found: Optional[int] = None
    
    # Error tracking
    failure_stage: Optional[FailureType] = None
    error_message: Optional[str] = None
    
    def __post_init__(self):
        if self.verilog_files is None:
            self.verilog_files = []

@dataclass
class SynthesisResult:
    """FABulous synthesis preprocessing result"""
    success: bool
    runtime: float
    verilog_file: Optional[str] = None
    error_message: Optional[str] = None

@dataclass
class ParameterResult:
    """Single parameter configuration result"""
    config_id: int
    success: bool
    runtime: float
    
    # Configuration parameters
    parameters: Dict[str, Any] = None
    
    # NextPNR analysis results
    placement_info: Optional[Dict[str, Any]] = None
    routing_info: Optional[Dict[str, Any]] = None
    
    # Error tracking
    failure_type: Optional[FailureType] = None
    error_message: Optional[str] = None
    
    def __post_init__(self):
        if self.parameters is None:
            self.parameters = {}

@dataclass
class CompletePipelineResult:
    """Complete end-to-end pipeline result"""
    source_file: str
    overall_success: bool
    total_runtime: float
    
    # Stage results
    verilog_pipeline: VerilogPipelineResult
    synthesis_result: SynthesisResult
    parameter_results: List[ParameterResult]
    
    # Overall failure tracking
    primary_failure_type: FailureType = FailureType.NONE
    
    def to_dict(self) -> Dict[str, Any]:
        """Convert to dictionary for DataFrame creation"""
        base_dict = {
            'source_file': os.path.basename(self.source_file),
            'overall_success': self.overall_success,
            'total_runtime': self.total_runtime,
            'primary_failure': self.primary_failure_type.value,
            
            # Verilog pipeline
            'verilog_success': self.verilog_pipeline.success,
            'verilog_runtime': self.verilog_pipeline.runtime,
            'mlir_opt_time': self.verilog_pipeline.mlir_optimization_time,
            'loop_extract_time': self.verilog_pipeline.loop_extraction_time,
            'futil_gen_time': self.verilog_pipeline.futil_generation_time,
            'loops_found': self.verilog_pipeline.loops_found,
            
            # Synthesis
            'synthesis_success': self.synthesis_result.success,
            'synthesis_runtime': self.synthesis_result.runtime,
            
            # Parameter sweep summary
            'total_configs': len(self.parameter_results),
            'successful_configs': sum(1 for p in self.parameter_results if p.success),
        }
        
        if self.parameter_results:
            successful_params = [p for p in self.parameter_results if p.success]
            if successful_params:
                base_dict['avg_param_runtime'] = sum(p.runtime for p in successful_params) / len(successful_params)
                base_dict['min_param_runtime'] = min(p.runtime for p in successful_params)
                base_dict['max_param_runtime'] = max(p.runtime for p in successful_params)
        
        return base_dict

In [ ]:
# Refactored NextPNR Analysis and Execution
# Clean, maintainable nextpnr log analysis and execution with proper stage separation

def analyze_nextpnr_logs(stdout: str, stderr: str) -> tuple[PlacementResult, RoutingResult]:
    """
    Analyze nextpnr output logs to extract placement and routing information.
    
    Parameters
    ----------
    stdout : str
        Standard output from nextpnr run
    stderr : str
        Standard error from nextpnr run
    
    Returns
    -------
    tuple[PlacementResult, RoutingResult]
        Tuple containing placement and routing analysis results
    """
    combined_output = stdout + "\n" + stderr
    
    # Analyze Placement Stage
    placement_success = "Final Placement" in combined_output
    placement_error = None
    
    placement_failure_patterns = [
        "ERROR: Unable to place cell",
        "ERROR: Placement failed", 
        "Error: Cannot place",
        "Error: Placement error",
        "FATAL: Placement failed",
        "failed to place"
    ]
    
    for pattern in placement_failure_patterns:
        if pattern.lower() in combined_output.lower():
            placement_success = False
            placement_error = pattern
            break
    
    placement_result = PlacementResult(
        status=StageStatus.SUCCESS if placement_success else StageStatus.FAILED,
        success=placement_success,
        error_details=placement_error
    )
    
    # Analyze Routing Stage 
    routing_failed = "ERROR: Max iteration count reached" in combined_output
    routing_error = None
    arc_queue_size = None
    
    # Extract arc_queue size
    arc_queue_matches = re.findall(r"current arc_queue size[:\s]*(\d+)", combined_output, re.IGNORECASE)
    if arc_queue_matches:
        arc_queue_size = int(arc_queue_matches[-1])  # Take the last (most recent) size
    
    # Check for routing error patterns
    routing_error_patterns = [
        "ERROR: Max iteration count reached",
        "ERROR: Routing failed",
        "Error: Cannot route", 
        "Error: Routing error",
        "FATAL: Routing failed"
    ]
    
    for pattern in routing_error_patterns:
        if pattern.lower() in combined_output.lower():
            routing_failed = True
            routing_error = pattern
            break
    
    # Routing is successful if placement succeeded and routing didn't explicitly fail
    routing_success = placement_success and not routing_failed
    
    routing_result = RoutingResult(
        status=StageStatus.SUCCESS if routing_success else StageStatus.FAILED,
        success=routing_success,
        failed=routing_failed,
        arc_queue_size=arc_queue_size,
        error_details=routing_error
    )
    
    return placement_result, routing_result

def determine_failure_type(synthesis: SynthesisResult, placement: PlacementResult, routing: RoutingResult, return_code: int) -> FailureType:
    """
    Determine the primary failure type based on stage results.
    
    Parameters
    ----------
    synthesis : SynthesisResult
        Synthesis stage result
    placement : PlacementResult  
        Placement stage result
    routing : RoutingResult
        Routing stage result
    return_code : int
        Process return code
        
    Returns
    -------
    FailureType
        Primary failure type classification
    """
    # Check synthesis first (happens before nextpnr)
    if synthesis.is_failed:
        return FailureType.SYNTHESIS
    
    # Check placement
    if not placement.success:
        return FailureType.PLACEMENT
    
    # Check routing
    if not routing.success or routing.failed:
        return FailureType.ROUTING
    
    # Check for timeout/exception based on return code
    if return_code == -1:
        return FailureType.TIMEOUT
    elif return_code < 0:
        return FailureType.EXCEPTION
    elif return_code != 0:
        return FailureType.UNKNOWN
    
    return FailureType.NONE

def run_nextpnr_with_parameters(
    connectivity_factor: float, 
    congestion_factor: float, 
    synthesis_result: SynthesisResult,
    source_file_name: str = "default"
) -> NextPNRResult:
    """
    Execute nextpnr-himbaechel with specified parameters and clean result structure.
    
    Parameters
    ----------
    connectivity_factor : float
        Placer heap architecture connectivity factor
    congestion_factor : float
        Placer heap congestion aware factor
    synthesis_result : SynthesisResult
        Previous synthesis stage result
    source_file_name : str
        Name of the source file (used for output naming)
    
    Returns
    -------
    NextPNRResult
        Complete nextpnr execution result with stage-by-stage analysis
    """
    from datetime import datetime
    import time
    
    # Generate unique output filenames
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    conn_str = f"{connectivity_factor:.1f}".replace(".", "p")
    cong_str = f"{congestion_factor:.1f}".replace(".", "p")
    clean_source_name = Path(source_file_name).stem.replace(".", "_")
    
    output_json = OUTPUT_DIR / f"{clean_source_name}_conn{conn_str}_cong{cong_str}.json"
    output_fasm = OUTPUT_DIR / f"{clean_source_name}_conn{conn_str}_cong{cong_str}.fasm"
    
    # Check if synthesis was successful - if not, skip nextpnr
    if synthesis_result.is_failed:
        # Create failed result without running nextpnr
        placement_result = PlacementResult(
            status=StageStatus.SKIPPED,
            success=False,
            error_details="Skipped due to synthesis failure"
        )
        routing_result = RoutingResult(
            status=StageStatus.SKIPPED,
            success=False,
            failed=False,
            error_details="Skipped due to synthesis failure"
        )
        
        return NextPNRResult(
            source_file=source_file_name,
            connectivity_factor=connectivity_factor,
            congestion_factor=congestion_factor,
            overall_success=False,
            runtime=0.0,
            return_code=-999,  # Special code for skipped
            stdout="",
            stderr="Skipped due to synthesis failure",
            timestamp=timestamp,
            synthesis=synthesis_result,
            placement=placement_result,
            routing=routing_result,
            failure_type=FailureType.SYNTHESIS,
            output_json=None,
            output_fasm=None
        )
    
    # Build nextpnr command using the synthesis JSON
    synth_json_path = Path(synthesis_result.synthesis_json_path)
    
    nextpnr_cmd = [
        "nextpnr-himbaechel",
        "--chipdb", str(CHIPDB_PATH),
        "--device", "FABulous", 
        "--json", str(synth_json_path),
        "--write", str(output_json),
        "-o", f"constrain-pair={CONSTRAIN_PAIR}",
        "-o", f"fdc={FDC_PATH}",
        "--placer-heap-seed-placement-strategy", "graph_grid",
        "--placer-heap-beta", str(BETA_VALUE),
        "--placer-heap-arch-connectivity-factor", str(connectivity_factor),
        "--placer-heap-congestion-aware-factor", str(congestion_factor),
        "-o", f"placeTrial={PLACE_TRIALS}",
        "--router1-timeout", str(ROUTER_TIMEOUT)
    ]
    
    start_time = time.time()
    
    try:
        print(f"Running nextpnr for {clean_source_name}: connectivity={connectivity_factor:.1f}, congestion={congestion_factor:.1f}")
        
        env = os.environ.copy()
        result = subprocess.run(nextpnr_cmd, capture_output=True, text=True, timeout=600, env=env)
        runtime = time.time() - start_time
        
        overall_success = result.returncode == 0
        
        # Analyze placement and routing stages
        placement_result, routing_result = analyze_nextpnr_logs(result.stdout, result.stderr)
        
        # Determine primary failure type
        failure_type = determine_failure_type(synthesis_result, placement_result, routing_result, result.returncode)
        
        return NextPNRResult(
            source_file=source_file_name,
            connectivity_factor=connectivity_factor,
            congestion_factor=congestion_factor,
            overall_success=overall_success,
            runtime=runtime,
            return_code=result.returncode,
            stdout=result.stdout,
            stderr=result.stderr,
            timestamp=timestamp,
            synthesis=synthesis_result,
            placement=placement_result,
            routing=routing_result,
            failure_type=failure_type,
            output_json=str(output_json) if overall_success else None,
            output_fasm=str(output_fasm) if overall_success else None
        )
        
    except subprocess.TimeoutExpired:
        runtime = time.time() - start_time
        
        placement_result = PlacementResult(
            status=StageStatus.TIMEOUT,
            success=False,
            error_details="Process timeout"
        )
        routing_result = RoutingResult(
            status=StageStatus.TIMEOUT,
            success=False,
            failed=False,
            error_details="Process timeout"
        )
        
        return NextPNRResult(
            source_file=source_file_name,
            connectivity_factor=connectivity_factor,
            congestion_factor=congestion_factor,
            overall_success=False,
            runtime=runtime,
            return_code=-1,
            stdout="",
            stderr="Timeout",
            timestamp=timestamp,
            synthesis=synthesis_result,
            placement=placement_result,
            routing=routing_result,
            failure_type=FailureType.TIMEOUT,
            output_json=None,
            output_fasm=None
        )
        
    except Exception as e:
        runtime = time.time() - start_time
        
        placement_result = PlacementResult(
            status=StageStatus.ERROR,
            success=False,
            error_details=f"Exception: {str(e)}"
        )
        routing_result = RoutingResult(
            status=StageStatus.ERROR,
            success=False,
            failed=False,
            error_details=f"Exception: {str(e)}"
        )
        
        return NextPNRResult(
            source_file=source_file_name,
            connectivity_factor=connectivity_factor,
            congestion_factor=congestion_factor,
            overall_success=False,
            runtime=runtime,
            return_code=-2,
            stdout="",
            stderr=str(e),
            timestamp=timestamp,
            synthesis=synthesis_result,
            placement=placement_result,
            routing=routing_result,
            failure_type=FailureType.EXCEPTION,
            output_json=None,
            output_fasm=None
        )

print("✅ Refactored nextpnr execution with clean stage separation")
print("✅ Added comprehensive log analysis with proper result structures")
print("✅ Implemented maintainable failure type classification")

In [ ]:
def runNextpnrHimbaechel(connectivityFactor: float, congestionFactor: float, sourceFileName: str = "default") -> dict:
    """
    Run nextpnr-himbaechel with specified parameters.
    
    Parameters
    ----------
    connectivityFactor : float
        Placer heap architecture connectivity factor
    congestionFactor : float
        Placer heap congestion aware factor
    sourceFileName : str
        Name of the source file (used for output naming)
    
    Returns
    -------
    dict
        Dictionary containing run results and comprehensive failure analysis
        Note: This function handles placement and routing only. 
        Synthesis success is determined earlier in the FABulous preprocessing stage.
    """
    # Generate unique output filenames with source file name
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    connStr = f"{connectivityFactor:.1f}".replace(".", "p")
    congStr = f"{congestionFactor:.1f}".replace(".", "p")
    
    # Clean source file name for filename
    cleanSourceName = Path(sourceFileName).stem.replace(".", "_")
    
    outputJson = OUTPUT_DIR / f"{cleanSourceName}_conn{connStr}_cong{congStr}.json"
    outputFasm = OUTPUT_DIR / f"{cleanSourceName}_conn{connStr}_cong{congStr}.fasm"
    
    # Build nextpnr command - use the synthesis JSON generated from the Verilog file
    synthJsonPath = VERILOG_OUTPUT_DIR / f"{Path(sourceFileName).stem}_synth.json"
    
    nextpnrCmd = [
        "nextpnr-himbaechel",
        "--chipdb", str(CHIPDB_PATH),
        "--device", "FABulous",
        "--json", str(synthJsonPath),
        "--write", str(outputJson),
        "-o", f"constrain-pair={CONSTRAIN_PAIR}",
        # "-o", f"fasm={outputFasm}",  # Uncomment if FASM output needed
        "-o", f"fdc={FDC_PATH}",
        "--placer-heap-seed-placement-strategy", "graph_grid",
        "--placer-heap-beta", str(BETA_VALUE),
        "--placer-heap-arch-connectivity-factor", str(connectivityFactor),
        "--placer-heap-congestion-aware-factor", str(congestionFactor),
        "-o", f"placeTrial={PLACE_TRIALS}",
        "--router1-timeout", str(ROUTER_TIMEOUT)
    ]
    
    startTime = time.time()
    
    try:
        print(f"Running nextpnr for {cleanSourceName}: connectivity={connectivityFactor:.1f}, congestion={congestionFactor:.1f}")
        
        # Set environment for subprocess
        env = os.environ.copy()
        result = subprocess.run(nextpnrCmd, capture_output=True, text=True, timeout=600, env=env)
        endTime = time.time()
        runtime = endTime - startTime
        
        success = result.returncode == 0
        
        # Analyze logs for detailed placement and routing failure information
        logAnalysis = analyzeNextpnrLogs(result.stdout, result.stderr)
        
        # Note: synthesisSuccess should be determined from FABulous preprocessing, not nextpnr logs
        # If we reach this point, synthesis was successful (otherwise we wouldn't have a valid JSON file)
        # The actual synthesis success is tracked separately in the preprocessing stage
        
        return {
            "sourceFileName": sourceFileName,
            "connectivityFactor": connectivityFactor,
            "congestionFactor": congestionFactor,
            "success": success,
            "runtime": runtime,
            "returnCode": result.returncode,
            "stdout": result.stdout,
            "stderr": result.stderr,
            "outputJson": str(outputJson) if success else None,
            "outputFasm": str(outputFasm) if success else None,
            "timestamp": timestamp,
            # Add detailed placement and routing failure analysis
            "placementSuccess": logAnalysis["placementSuccess"],
            "routingSuccess": logAnalysis["routingSuccess"], 
            "routingFailed": logAnalysis["routingFailed"],
            "arcQueueSize": logAnalysis["arcQueueSize"],
            "failureType": logAnalysis["failureType"],
            "errorDetails": logAnalysis["errorDetails"],
            # Note: synthesisSuccess is handled separately in preprocessing stage
            # If this function is called, synthesis preprocessing was successful
            "synthesisSuccess": True,  # By reaching this point, synthesis JSON exists and is valid
            "synthesisError": "none"   # No synthesis errors if we got to nextpnr stage
        }
        
    except subprocess.TimeoutExpired:
        endTime = time.time()
        runtime = endTime - startTime
        logger.warning(f"nextpnr timed out for {cleanSourceName}: connectivity={connectivityFactor:.1f}, congestion={congestionFactor:.1f}")
        
        return {
            "sourceFileName": sourceFileName,
            "connectivityFactor": connectivityFactor,
            "congestionFactor": congestionFactor,
            "success": False,
            "runtime": runtime,
            "returnCode": -1,
            "stdout": "",
            "stderr": "Timeout",
            "outputJson": None,
            "outputFasm": None,
            "timestamp": timestamp,
            # Add failure analysis fields for timeout case
            "placementSuccess": False,
            "routingSuccess": False,
            "routingFailed": False,
            "arcQueueSize": None,
            "failureType": "timeout",
            "errorDetails": "Process timeout",
            "synthesisSuccess": True,  # Synthesis was successful if we got to nextpnr
            "synthesisError": "none"
        }
    except Exception as e:
        endTime = time.time()
        runtime = endTime - startTime
        print(f"Error running nextpnr for {cleanSourceName}: {e}")
        
        return {
            "sourceFileName": sourceFileName,
            "connectivityFactor": connectivityFactor,
            "congestionFactor": congestionFactor,
            "success": False,
            "runtime": runtime,
            "returnCode": -2,
            "stdout": "",
            "stderr": str(e),
            "outputJson": None,
            "outputFasm": None,
            "timestamp": timestamp,
            # Add failure analysis fields for exception case
            "placementSuccess": False,
            "routingSuccess": False,
            "routingFailed": False,
            "arcQueueSize": None,
            "failureType": "exception",
            "errorDetails": f"Exception: {str(e)}",
            "synthesisSuccess": True,  # Synthesis was successful if we got to nextpnr
            "synthesisError": "none"
        }

In [ ]:
# Complete Pipeline: MLIR → Verilog → Synthesis → Parameter Sweep
# Integrated, maintainable pipeline with comprehensive stage tracking

def run_complete_mlir_pipeline(mlir_src_path: Path) -> CompletePipelineResult:
    """
    Run the complete MLIR → Verilog → Synthesis → Parameter Sweep pipeline.
    
    Parameters
    ----------
    mlir_src_path : Path
        Path to the source MLIR file
        
    Returns
    -------
    CompletePipelineResult
        Complete pipeline result with comprehensive tracking
    """
    import time
    total_pipeline_start = time.time()
    
    print(f"🚀 Starting Complete MLIR Pipeline: {mlir_src_path.name}")
    print("=" * 80)
    
    # Stage 1: MLIR → Verilog Generation
    print(f"🔧 Pipeline Stage 1: MLIR → Verilog Generation")
    verilog_generation_result = run_complete_verilog_generation(mlir_src_path)
    
    if not verilog_generation_result.overall_success:
        print(f"❌ Verilog generation failed: {verilog_generation_result.primary_failure_type.value}")
        return CompletePipelineResult(
            source_mlir_file=mlir_src_path.name,
            verilog_generation=verilog_generation_result,
            synthesis_result=SynthesisResult(
                status=StageStatus.SKIPPED,
                source_file=mlir_src_path.name,
                error_message="Skipped due to Verilog generation failure"
            ),
            parameter_results=[],
            overall_success=False,
            total_pipeline_runtime=time.time() - total_pipeline_start
        )
    
    print(f"✅ Verilog generation completed: {len(verilog_generation_result.verilog_files)} files generated")
    
    # For now, we'll process the first generated Verilog file for parameter sweep
    # In the future, this could be extended to handle multiple files
    if not verilog_generation_result.verilog_files:
        print(f"❌ No Verilog files generated")
        return CompletePipelineResult(
            source_mlir_file=mlir_src_path.name,
            verilog_generation=verilog_generation_result,
            synthesis_result=SynthesisResult(
                status=StageStatus.FAILED,
                source_file=mlir_src_path.name,
                error_type="no_verilog_files",
                error_message="No Verilog files were generated"
            ),
            parameter_results=[],
            overall_success=False,
            total_pipeline_runtime=time.time() - total_pipeline_start
        )
    
    # Use the first Verilog file for parameter sweep
    primary_verilog_file = Path(verilog_generation_result.verilog_files[0])
    
    # Stage 2: Synthesis Preprocessing
    print(f"🔧 Pipeline Stage 2: Synthesis Preprocessing")
    synthesis_result = run_fabulous_synthesis(primary_verilog_file)
    
    if synthesis_result.is_failed:
        print(f"❌ Synthesis failed: {synthesis_result.error_type}")
        return CompletePipelineResult(
            source_mlir_file=mlir_src_path.name,
            verilog_generation=verilog_generation_result,
            synthesis_result=synthesis_result,
            parameter_results=[],
            overall_success=False,
            total_pipeline_runtime=time.time() - total_pipeline_start
        )
    
    print(f"✅ Synthesis completed successfully")
    
    # Stage 3: Parameter Sweep
    print(f"🔧 Pipeline Stage 3: Parameter Sweep")
    print(f"📊 Running {len(grid)} parameter combinations...")
    
    parameter_results: List[NextPNRResult] = []
    
    def run_single_parameter_combination(args: tuple) -> NextPNRResult:
        """Run a single parameter combination with logging"""
        connectivity_factor, congestion_factor = args
        
        result = run_nextpnr_with_parameters(
            connectivity_factor=connectivity_factor,
            congestion_factor=congestion_factor, 
            synthesis_result=synthesis_result,
            source_file_name=primary_verilog_file.name
        )
        
        # Log result with clean stage summary
        if result.overall_success:
            print(f"✅ conn={connectivity_factor:.1f}, cong={congestion_factor:.1f} - {result.stage_summary} ({result.runtime:.2f}s)")
        else:
            print(f"❌ conn={connectivity_factor:.1f}, cong={congestion_factor:.1f} - {result.failure_type.value} ({result.stage_summary})")
        
        return result

    # Execute parameter combinations with controlled concurrency
    max_workers = min(16, os.cpu_count() or 2)
    
    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = [executor.submit(run_single_parameter_combination, args) for args in grid]
        for future in concurrent.futures.as_completed(futures):
            parameter_results.append(future.result())
    
    # Calculate overall success
    successful_parameters = sum(1 for r in parameter_results if r.overall_success)
    overall_success = successful_parameters > 0
    total_pipeline_runtime = time.time() - total_pipeline_start
    
    result = CompletePipelineResult(
        source_mlir_file=mlir_src_path.name,
        verilog_generation=verilog_generation_result,
        synthesis_result=synthesis_result,
        parameter_results=parameter_results,
        overall_success=overall_success,
        total_pipeline_runtime=total_pipeline_runtime
    )
    
    # Log comprehensive summary
    print(f"\n📊 Complete Pipeline Summary for {mlir_src_path.name}:")
    print(f"   🔧 Verilog Generation: {verilog_generation_result.stage_summary}")
    print(f"   🔧 Synthesis: {'✅' if synthesis_result.is_success else '❌'}")
    print(f"   📊 Parameter Sweep: {successful_parameters}/{len(parameter_results)} successful")
    print(f"   ⚡ Total Pipeline Runtime: {total_pipeline_runtime:.2f}s")
    print(f"   🎯 Overall Success: {'✅' if overall_success else '❌'}")
    
    return result

def process_mlir_benchmarks_with_complete_pipeline() -> tuple[List[CompletePipelineResult], pd.DataFrame, pd.DataFrame]:
    """
    Process all MLIR benchmarks through the complete pipeline.
    
    Returns
    -------
    tuple[List[CompletePipelineResult], pd.DataFrame, pd.DataFrame]
        Complete pipeline results, Verilog generation DataFrame, and Parameter sweep DataFrame
    """
    print(f"🚀 Starting Complete MLIR Pipeline Processing")
    print(f"📁 Processing MLIR files from: {BENCHMARK_ROOT_DIR}")
    print("=" * 80)
    
    # Find all MLIR files
    mlir_files = list(BENCHMARK_ROOT_DIR.glob("*.mlir"))
    if not mlir_files:
        print(f"❌ No MLIR files found in {BENCHMARK_ROOT_DIR}")
        return [], pd.DataFrame(), pd.DataFrame()
    
    print(f"📊 Found {len(mlir_files)} MLIR files to process")
    
    # Process each MLIR file through the complete pipeline
    complete_results: List[CompletePipelineResult] = []
    all_verilog_results = []
    all_parameter_results = []
    
    for file_index, mlir_file in enumerate(mlir_files, 1):
        print(f"\n📄 [{file_index}/{len(mlir_files)}] Processing: {mlir_file.name}")
        print("-" * 60)
        
        # Run complete pipeline
        pipeline_result = run_complete_mlir_pipeline(mlir_file)
        complete_results.append(pipeline_result)
        
        # Collect Verilog generation results
        verilog_result_dict = pipeline_result.verilog_generation.to_dict()
        verilog_result_dict['sourceMlirFile'] = mlir_file.name
        all_verilog_results.append(verilog_result_dict)
        
        # Collect parameter results with metadata
        for param_result in pipeline_result.parameter_results:
            param_result_dict = param_result.to_dict()
            param_result_dict['sourceMlirFile'] = mlir_file.name
            param_result_dict['verilogGenerationSuccess'] = pipeline_result.verilog_generation.overall_success
            param_result_dict['totalPipelineRuntime'] = pipeline_result.total_pipeline_runtime
            all_parameter_results.append(param_result_dict)
        
        # Brief pause between files
        if file_index < len(mlir_files):
            time.sleep(2)
    
    # Create DataFrames
    verilog_df = pd.DataFrame(all_verilog_results) if all_verilog_results else pd.DataFrame()
    parameter_df = pd.DataFrame(all_parameter_results) if all_parameter_results else pd.DataFrame()
    
    # Print overall summary
    print(f"\n" + "=" * 80)
    print(f"🎯 COMPLETE PIPELINE SUMMARY")
    print("=" * 80)
    
    total_files = len(complete_results)
    verilog_successful = sum(1 for r in complete_results if r.verilog_generation.overall_success)
    synthesis_successful = sum(1 for r in complete_results if r.synthesis_result.is_success)
    parameter_successful = sum(1 for r in complete_results if r.overall_success)
    
    print(f"📊 Processing Statistics:")
    print(f"   📁 Total MLIR files: {total_files}")
    print(f"   🔧 Verilog generation successful: {verilog_successful}/{total_files} ({verilog_successful/total_files*100:.1f}%)")
    print(f"   🔧 Synthesis successful: {synthesis_successful}/{total_files} ({synthesis_successful/total_files*100:.1f}%)")
    print(f"   📊 Parameter sweep successful: {parameter_successful}/{total_files} ({parameter_successful/total_files*100:.1f}%)")
    
    if len(parameter_df) > 0:
        total_param_runs = len(parameter_df)
        successful_param_runs = parameter_df['success'].sum()
        print(f"   🎯 Total parameter runs: {successful_param_runs}/{total_param_runs} ({successful_param_runs/total_param_runs*100:.1f}%)")
    
    # Failure analysis
    if verilog_successful < total_files:
        print(f"\n🔍 Verilog Generation Failures:")
        verilog_failures = [r for r in complete_results if not r.verilog_generation.overall_success]
        failure_types = {}
        for failure in verilog_failures:
            failure_type = failure.verilog_generation.primary_failure_type.value
            failure_types[failure_type] = failure_types.get(failure_type, 0) + 1
        
        for failure_type, count in failure_types.items():
            print(f"      • {failure_type}: {count}")
    
    print(f"\n📋 DataFrames Created:")
    print(f"   🔧 Verilog Generation: {len(verilog_df)} rows")
    print(f"   📊 Parameter Sweep: {len(parameter_df)} rows")
    
    return complete_results, verilog_df, parameter_df

def create_benchmark_metadata_for_mlir(mlir_file: str) -> Dict[str, str]:
    """Create metadata for MLIR-based benchmark tracking"""
    benchmark_name = Path(mlir_file).stem
    
    return {
        "benchmarkName": benchmark_name,
        "mlirSource": mlir_file,
        "pipelineType": "complete_mlir_pipeline"
    }

print("✅ Integrated complete MLIR → Verilog → Synthesis → Parameter pipeline")
print("✅ Added comprehensive result tracking across all stages")
print("✅ Implemented maintainable pipeline orchestration with full visibility")

In [15]:
# Benchmark Selection Configuration
# Leave empty to process all benchmarks, or specify a list of benchmark names to process only those

# Examples:
# selectedBenchmarks = []  # Process all benchmarks (default)
# selectedBenchmarks = ["aes_aes", "bfs_bulk"]  # Process only these benchmarks
# selectedBenchmarks = [name for name in benchmarkStructure.keys() if "fft" in name]  # Filter by name pattern

selectedBenchmarks = []  # Default: process all benchmarks

# Apply benchmark selection filter
if selectedBenchmarks:
    print(f"Filtering to {len(selectedBenchmarks)} selected benchmarks:")
    filteredBenchmarkStructure = {}
    filteredVerilogFiles = []
    
    for benchmarkName in selectedBenchmarks:
        if benchmarkName in benchmarkStructure:
            filteredBenchmarkStructure[benchmarkName] = benchmarkStructure[benchmarkName]
            filteredVerilogFiles.extend(benchmarkStructure[benchmarkName])
            print(f"  - {benchmarkName}: {len(benchmarkStructure[benchmarkName])} files")
        else:
            logger.warning(f"  - {benchmarkName}: NOT FOUND")
    
    # Update the structures to use
    benchmarkStructure = filteredBenchmarkStructure  
    availableVerilogFiles = filteredVerilogFiles
    
    if not filteredVerilogFiles:
        logger.warning("No files matched the selection criteria!")
else:
    print(f"Will process all {len(benchmarkStructure)} benchmarks with {len(availableVerilogFiles)} total files")

# Final check
if not availableVerilogFiles:
    logger.warning("No Verilog files available for processing!")
    if VERILOG_OUTPUT_DIR.exists():
        print(f"Contents of {VERILOG_OUTPUT_DIR}:")
        for item in VERILOG_OUTPUT_DIR.iterdir():
            print(f"  - {item.name}")
    else:
        print(f"Directory {VERILOG_OUTPUT_DIR} does not exist!")
        
print(f"Final selection: {len(benchmarkStructure)} benchmarks, {len(availableVerilogFiles)} files to process")

Will process all 19 benchmarks with 37 total files
Final selection: 19 benchmarks, 37 files to process


In [ ]:
# Main execution with unified pipeline
mlir_files = [
    "/home/kelvin/FABulous_fork/benchmarks/example1/example1.mlir",
    "/home/kelvin/FABulous_fork/benchmarks/example2/example2.mlir",
    "/home/kelvin/FABulous_fork/benchmarks/example3/example3.mlir"
]

# Run complete pipeline for all MLIR files
complete_results = process_mlir_benchmarks(mlir_files)

print(f"\n=== Complete Pipeline Results ===")
print(f"Total files processed: {len(complete_results)}")

# Display summary statistics
successful_complete = sum(1 for r in complete_results if r.overall_success)
print(f"Successfully completed full pipeline: {successful_complete}/{len(complete_results)}")

# Stage-wise success rates
verilog_success = sum(1 for r in complete_results if r.verilog_pipeline.success)
synthesis_success = sum(1 for r in complete_results if r.synthesis_result.success)
pnr_success = sum(1 for r in complete_results if any(p.success for p in r.parameter_results))

print(f"Verilog generation success: {verilog_success}/{len(complete_results)}")
print(f"Synthesis success: {synthesis_success}/{len(complete_results)}")
print(f"Parameter sweep success: {pnr_success}/{len(complete_results)}")

# Display detailed results for each file
for result in complete_results:
    print(f"\n--- {os.path.basename(result.source_file)} ---")
    print(f"Overall success: {result.overall_success}")
    print(f"Primary failure: {result.primary_failure_type.value}")
    print(f"Total runtime: {result.total_runtime:.2f}s")
    
    # Stage details
    vp = result.verilog_pipeline
    print(f"  Verilog Pipeline: {'✓' if vp.success else '✗'} ({vp.runtime:.2f}s)")
    print(f"    MLIR Opt: {vp.mlir_optimization_time:.2f}s")
    print(f"    Loop Extract: {vp.loop_extraction_time:.2f}s") 
    print(f"    FUTIL Gen: {vp.futil_generation_time:.2f}s")
    if vp.loops_found is not None:
        print(f"    Loops found: {vp.loops_found}")
    
    print(f"  Synthesis: {'✓' if result.synthesis_result.success else '✗'} ({result.synthesis_result.runtime:.2f}s)")
    
    if result.parameter_results:
        successful_params = sum(1 for p in result.parameter_results if p.success)
        print(f"  Parameter sweep: {successful_params}/{len(result.parameter_results)} configs")

## 7. Interactive Dashboard

Interactive dashboard for exploring results across different Verilog files with combined analysis and file-specific views.

In [ ]:
# Enhanced Interactive Dashboard for Unified Pipeline
import ipywidgets as widgets
from IPython.display import display, clear_output
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

def create_pipeline_dashboard(complete_results: List[CompletePipelineResult]):
    """Enhanced dashboard for unified pipeline analysis"""
    
    # Create UI components
    file_selector = widgets.Dropdown(
        options=[(os.path.basename(r.source_file), i) for i, r in enumerate(complete_results)],
        description='MLIR File:',
        style={'description_width': 'initial'}
    )
    
    view_selector = widgets.Dropdown(
        options=[
            ('Pipeline Overview', 'overview'),
            ('Stage Details', 'stages'),
            ('Parameter Analysis', 'parameters'),
            ('Performance Metrics', 'performance'),
            ('Failure Analysis', 'failures')
        ],
        description='View:',
        value='overview'
    )
    
    output_area = widgets.Output()
    
    def update_display(change=None):
        with output_area:
            clear_output(wait=True)
            
            selected_idx = file_selector.value
            view_type = view_selector.value
            result = complete_results[selected_idx]
            
            if view_type == 'overview':
                display_pipeline_overview(result)
            elif view_type == 'stages':
                display_stage_details(result)
            elif view_type == 'parameters':
                display_parameter_analysis(result)
            elif view_type == 'performance':
                display_performance_metrics(result)
            elif view_type == 'failures':
                display_failure_analysis(result)
    
    # Connect event handlers
    file_selector.observe(update_display, names='value')
    view_selector.observe(update_display, names='value')
    
    # Display dashboard
    dashboard = widgets.VBox([
        widgets.HBox([file_selector, view_selector]),
        output_area
    ])
    display(dashboard)
    
    # Initial display
    update_display()

def display_pipeline_overview(result: CompletePipelineResult):
    """Display high-level pipeline overview"""
    print(f"=== Pipeline Overview: {os.path.basename(result.source_file)} ===")
    print(f"Overall Success: {'✓ PASS' if result.overall_success else '✗ FAIL'}")
    print(f"Primary Failure Type: {result.primary_failure_type.value}")
    print(f"Total Runtime: {result.total_runtime:.2f}s")
    print()
    
    # Stage success summary
    vp = result.verilog_pipeline
    stages = [
        ("Verilog Pipeline", vp.success),
        ("Synthesis", result.synthesis_result.success),
    ]
    
    print("Stage Success Summary:")
    for stage_name, success in stages:
        status = "✓ PASS" if success else "✗ FAIL"
        print(f"  {stage_name:20} {status}")
    
    if result.parameter_results:
        successful_params = sum(1 for p in result.parameter_results if p.success)
        print(f"  Parameter Sweep     {successful_params}/{len(result.parameter_results)} configs passed")
    
    # Detailed Verilog pipeline breakdown
    if not vp.success and vp.failure_stage:
        print(f"\nVerilog Pipeline failed at: {vp.failure_stage.value}")
    else:
        print(f"\nVerilog Pipeline stages:")
        print(f"  MLIR Optimization: {vp.mlir_optimization_time:.2f}s")
        print(f"  Loop Extraction: {vp.loop_extraction_time:.2f}s")
        print(f"  FUTIL Generation: {vp.futil_generation_time:.2f}s")
        if vp.loops_found is not None:
            print(f"  Loops found: {vp.loops_found}")
    
    # Create visual pipeline flow
    fig, ax = plt.subplots(1, 1, figsize=(10, 3))
    stage_names = [s[0] for s in stages]
    stage_status = [1 if s[1] else 0 for s in stages]
    
    colors = ['green' if s else 'red' for s in stage_status]
    bars = ax.barh(range(len(stage_names)), [1]*len(stage_names), color=colors, alpha=0.7)
    
    ax.set_yticks(range(len(stage_names)))
    ax.set_yticklabels(stage_names)
    ax.set_xlim(0, 1)
    ax.set_xlabel('Pipeline Flow')
    ax.set_title(f'Pipeline Stages: {os.path.basename(result.source_file)}')
    
    for i, (bar, success) in enumerate(zip(bars, stage_status)):
        ax.text(0.5, i, '✓' if success else '✗', ha='center', va='center', 
                fontsize=12, fontweight='bold', color='white')
    
    plt.tight_layout()
    plt.show()

def display_stage_details(result: CompletePipelineResult):
    """Display detailed information for each stage"""
    print(f"=== Stage Details: {os.path.basename(result.source_file)} ===")
    
    vp = result.verilog_pipeline
    
    # Verilog Pipeline Details
    print("1. Verilog Generation Pipeline:")
    print(f"   Overall Success: {'✓' if vp.success else '✗'}")
    print(f"   Total Runtime: {vp.runtime:.2f}s")
    print(f"   MLIR Optimization: {vp.mlir_optimization_time:.2f}s")
    print(f"   Loop Extraction: {vp.loop_extraction_time:.2f}s")
    print(f"   FUTIL Generation: {vp.futil_generation_time:.2f}s")
    
    if vp.loops_found is not None:
        print(f"   Loops Found: {vp.loops_found}")
    if vp.optimized_mlir_file:
        print(f"   Optimized MLIR: {os.path.basename(vp.optimized_mlir_file)}")
    if vp.futil_file:
        print(f"   FUTIL File: {os.path.basename(vp.futil_file)}")
    if vp.verilog_files:
        print(f"   Verilog Files: {len(vp.verilog_files)} generated")
        for vf in vp.verilog_files:
            print(f"     - {os.path.basename(vf)}")
    
    if not vp.success:
        print(f"   Failure Stage: {vp.failure_stage.value if vp.failure_stage else 'unknown'}")
        if vp.error_message:
            print(f"   Error: {vp.error_message}")
    print()
    
    # Synthesis Details
    print("2. Synthesis (FABulous):")
    print(f"   Success: {'✓' if result.synthesis_result.success else '✗'}")
    print(f"   Runtime: {result.synthesis_result.runtime:.2f}s")
    if result.synthesis_result.verilog_file:
        print(f"   Processed Verilog: {os.path.basename(result.synthesis_result.verilog_file)}")
    if result.synthesis_result.error_message:
        print(f"   Error: {result.synthesis_result.error_message}")

def display_parameter_analysis(result: CompletePipelineResult):
    """Display parameter sweep analysis"""
    if not result.parameter_results:
        print("No parameter sweep results available")
        return
        
    print(f"=== Parameter Analysis: {os.path.basename(result.source_file)} ===")
    
    # Convert to DataFrame for analysis
    param_data = []
    for param_result in result.parameter_results:
        row = {
            'config_id': param_result.config_id,
            'success': param_result.success,
            'runtime': param_result.runtime,
            'failure_type': param_result.failure_type.value if param_result.failure_type else 'none'
        }
        
        # Add placement/routing metrics if available
        if param_result.placement_info:
            row.update(param_result.placement_info)
        if param_result.routing_info:
            row.update(param_result.routing_info)
            
        param_data.append(row)
    
    df = pd.DataFrame(param_data)
    
    # Summary statistics
    total_configs = len(df)
    successful_configs = df['success'].sum()
    print(f"Total Configurations: {total_configs}")
    print(f"Successful Configurations: {successful_configs}")
    print(f"Success Rate: {successful_configs/total_configs*100:.1f}%")
    print()
    
    # Failure analysis
    if successful_configs < total_configs:
        print("Failure Types:")
        failure_counts = df[~df['success']]['failure_type'].value_counts()
        for failure_type, count in failure_counts.items():
            print(f"  {failure_type}: {count}")
        print()
    
    # Performance analysis for successful configs
    if successful_configs > 0:
        successful_df = df[df['success']]
        print("Performance Statistics (Successful Configs):")
        print(f"  Average Runtime: {successful_df['runtime'].mean():.2f}s")
        print(f"  Runtime Range: {successful_df['runtime'].min():.2f}s - {successful_df['runtime'].max():.2f}s")
        
        # Plot runtime distribution
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
        
        # Success rate pie chart
        ax1.pie([successful_configs, total_configs - successful_configs], 
                labels=['Success', 'Failure'], autopct='%1.1f%%',
                colors=['green', 'red'], alpha=0.7)
        ax1.set_title('Configuration Success Rate')
        
        # Runtime histogram
        if successful_configs > 1:
            ax2.hist(successful_df['runtime'], bins=min(10, successful_configs), 
                    alpha=0.7, color='blue')
            ax2.set_xlabel('Runtime (seconds)')
            ax2.set_ylabel('Count')
            ax2.set_title('Runtime Distribution (Successful Configs)')
        
        plt.tight_layout()
        plt.show()

def display_performance_metrics(result: CompletePipelineResult):
    """Display performance metrics across all stages"""
    print(f"=== Performance Metrics: {os.path.basename(result.source_file)} ===")
    
    vp = result.verilog_pipeline
    
    # Collect stage runtimes
    stage_runtimes = {
        'MLIR Optimization': vp.mlir_optimization_time,
        'Loop Extraction': vp.loop_extraction_time,
        'FUTIL Generation': vp.futil_generation_time,
        'Synthesis': result.synthesis_result.runtime,
    }
    
    if result.parameter_results:
        param_runtimes = [p.runtime for p in result.parameter_results if p.success]
        if param_runtimes:
            stage_runtimes['Parameter Sweep (avg)'] = sum(param_runtimes) / len(param_runtimes)
    
    print("Stage Runtimes:")
    total_time = sum(stage_runtimes.values())
    for stage, runtime in stage_runtimes.items():
        percentage = (runtime / total_time) * 100 if total_time > 0 else 0
        print(f"  {stage:25} {runtime:6.2f}s ({percentage:4.1f}%)")
    
    print(f"\nTotal Pipeline Time: {result.total_runtime:.2f}s")
    
    # Create performance visualization
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
    
    # Stage runtime bar chart
    stages = list(stage_runtimes.keys())
    runtimes = list(stage_runtimes.values())
    
    bars = ax1.bar(range(len(stages)), runtimes, alpha=0.7)
    ax1.set_xticks(range(len(stages)))
    ax1.set_xticklabels(stages, rotation=45, ha='right')
    ax1.set_ylabel('Runtime (seconds)')
    ax1.set_title('Stage Runtimes')
    
    # Add value labels on bars
    for bar, runtime in zip(bars, runtimes):
        height = bar.get_height()
        ax1.text(bar.get_x() + bar.get_width()/2., height,
                f'{runtime:.2f}s', ha='center', va='bottom')
    
    # Runtime pie chart
    ax2.pie(runtimes, labels=stages, autopct='%1.1f%%', startangle=90)
    ax2.set_title('Runtime Distribution')
    
    plt.tight_layout()
    plt.show()

def display_failure_analysis(result: CompletePipelineResult):
    """Display detailed failure analysis"""
    print(f"=== Failure Analysis: {os.path.basename(result.source_file)} ===")
    
    if result.overall_success:
        print("✓ Pipeline completed successfully - no failures to analyze")
        return
    
    print(f"Primary Failure Type: {result.primary_failure_type.value}")
    print()
    
    # Verilog pipeline failure analysis
    vp = result.verilog_pipeline
    if not vp.success:
        print("Verilog Pipeline Failure:")
        print(f"  Failed at stage: {vp.failure_stage.value if vp.failure_stage else 'unknown'}")
        if vp.error_message:
            print(f"  Error: {vp.error_message}")
        print(f"  Runtime before failure: {vp.runtime:.2f}s")
        print()
    
    # Synthesis failure analysis
    if not result.synthesis_result.success:
        print("Synthesis Failure:")
        if result.synthesis_result.error_message:
            print(f"  Error: {result.synthesis_result.error_message}")
        print(f"  Runtime: {result.synthesis_result.runtime:.2f}s")
        print()
    
    # Parameter sweep failures
    if result.parameter_results:
        failed_params = [p for p in result.parameter_results if not p.success]
        if failed_params:
            print(f"Parameter Sweep Failures: {len(failed_params)}/{len(result.parameter_results)} configs")
            
            # Group by failure type
            failure_groups = {}
            for param in failed_params:
                failure_type = param.failure_type.value if param.failure_type else 'unknown'
                if failure_type not in failure_groups:
                    failure_groups[failure_type] = []
                failure_groups[failure_type].append(param)
            
            for failure_type, params in failure_groups.items():
                print(f"  {failure_type}: {len(params)} configs")
                if params[0].error_message:
                    print(f"    Example error: {params[0].error_message[:100]}...")

# Create the enhanced dashboard
if 'complete_results' in globals() and complete_results:
    print("Creating Enhanced Pipeline Dashboard...")
    create_pipeline_dashboard(complete_results)
else:
    print("No complete pipeline results available. Run the pipeline first.")